### Step 1 : step-by-step description for the Code: 4company_combo_start_day_qty

1) Prompt for inputs — ask for input Excel path, output Excel path, INVESTMENT (float), COMBO_SIZE (int), OBSERVATION_DAYS (int), MONTH_ROLLING (months to skip), ROLL_TYPE (year/month/week/day) and INCREMENT (integer step).

2) Load Excel — read the input file into a DataFrame and strip column name whitespace.

3) Detect date column — auto-find the first column whose name contains "date", parse it to datetime (day-first), drop invalid dates and sort rows by date.

4) Detect company columns — treat every other numeric column as a company price series and build company_names.

5) Compute start date — set start_date = earliest date + MONTH_ROLLING months; set current to the first day of that month. Print dataset start and processing start.

6) Rolling-step helper — get_next_date(current) advances current by INCREMENT units according to ROLL_TYPE (year/month/week/day).

7) Open Excel writer — create an openpyxl ExcelWriter to save results into multiple sheets inside the specified output_file.

8) Main rolling loop — while current is ≤ last date in data:

* set observe_start = current and observe_end = current + OBSERVATION_DAYS.

* slice data into the observation window; if empty, advance current using get_next_date and continue.

9) Pick calculation date — for a non-empty window, choose the earliest date inside it (selected_date) and take that row as the price reference.

10) Generate combinations — iterate all combinations of COMBO_SIZE companies from company_names.

11) Compute quantities — for each combo, compute INVESTMENT / price for each company (round to 2 decimals). If price is NaN or 0, put "NA".

12) Assemble results — collect rows as [Combo (comma list), Quantities (comma list), Calculation_Date] and form a DataFrame out_df.

13) Write sheet — write out_df to the Excel writer as a sheet named YYYY-MM (from current).

14) Advance roll — update current = get_next_date(current) and repeat until past the last date.

15) Finish — close the writer (file saved) and print success message with the output file path.

In [2]:
#4company_combo_start_day_qty

import pandas as pd
from itertools import combinations

# ==========================================================
#                    USER INPUTS
# ==========================================================

input_file = input("\nEnter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

def get_float(prompt):
    while True:
        try:
            return float(input(prompt).strip())
        except:
            print("Invalid value! Enter numeric value.")

def get_int(prompt):
    while True:
        try:
            return int(input(prompt).strip())
        except:
            print("Invalid value! Enter integer.")

INVESTMENT = get_float("\nEnter investment amount per company: ")
COMBO_SIZE = get_int("Enter number of companies per combination: ")
OBSERVATION_DAYS = get_int("Enter number of observation days: ")

ROLLING_START_DATE = input(
    "Enter rolling start date (YYYY-MM-DD), example 2020-04-01: "
).strip()

while True:
    ROLL_TYPE = input("Enter rolling type (year/month/week/day): ").strip().lower()
    if ROLL_TYPE in ["year", "month", "week", "day"]:
        break
    print("Invalid rolling type!")

INCREMENT = get_int("Enter rolling increment (example: 1): ")

# ==========================================================
#                     LOAD DATA
# ==========================================================

print("\nLoading file...")
df = pd.read_excel(input_file)
df.columns = df.columns.str.strip()

date_col = [c for c in df.columns if "date" in c.lower()][0]
df[date_col] = pd.to_datetime(df[date_col], dayfirst=True, errors="coerce")
df = df.dropna(subset=[date_col]).sort_values(date_col)

company_names = [
    c for c in df.columns
    if c != date_col and pd.api.types.is_numeric_dtype(df[c])
]

print(f"Dataset start date : {df[date_col].min().date()}")
print(f"Detected companies : {len(company_names)}")

# ==========================================================
#           FIXED ROLLING START DATE (NO MONDAY SHIFT)
# ==========================================================

current = pd.to_datetime(ROLLING_START_DATE)

print(f"Rolling begins from : {current.date()}")

# ==========================================================
#          ROLLING DATE FUNCTION
# ==========================================================

def get_next_date(dt):
    if ROLL_TYPE == "year":
        return dt + pd.DateOffset(years=INCREMENT)
    if ROLL_TYPE == "month":
        return dt + pd.DateOffset(months=INCREMENT)
    if ROLL_TYPE == "week":
        return dt + pd.DateOffset(weeks=INCREMENT)
    if ROLL_TYPE == "day":
        return dt + pd.DateOffset(days=INCREMENT)

# ==========================================================
#               WRITE OUTPUT
# ==========================================================

week_counter = 1
last_date = df[date_col].max()

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:

    while current <= last_date:

        observe_start = current
        observe_end = current + pd.Timedelta(days=OBSERVATION_DAYS)

        data = df[
            (df[date_col] >= observe_start) &
            (df[date_col] <= observe_end)
        ]

        if data.empty:
            current = get_next_date(current)
            continue

        selected_date = data[date_col].min()
        row = data[data[date_col] == selected_date].iloc[0]

        print(f"Writing Sheet: Week{week_counter} | Start: {observe_start.date()}")

        output_rows = []

        for combo in combinations(company_names, COMBO_SIZE):
            qty = []
            for c in combo:
                price = row[c]
                qty.append(
                    "NA" if pd.isna(price) or price == 0
                    else f"{INVESTMENT / price:.2f}"
                )

            output_rows.append([
                ",".join(combo),
                ",".join(qty),
                selected_date.date()
            ])

        out_df = pd.DataFrame(
            output_rows,
            columns=["Combo", "Quantities", "Calculation_Date"]
        )

        out_df.to_excel(
            writer,
            sheet_name=f"Week{week_counter}",
            index=False
        )

        week_counter += 1
        current = get_next_date(current)

print("\nPROCESS COMPLETED SUCCESSFULLY!")
print("Output saved at:", output_file)



Enter input Excel file path: C:\swarupa\Trading Stratergies\50 comp data (input file).xlsx
Enter output Excel file path: D:\week_qty\4company_weekwise_qty.xlsx

Enter investment amount per company: 50000
Enter number of companies per combination: 4
Enter number of observation days: 45
Enter rolling start date (YYYY-MM-DD), example 2020-04-01: 2020-04-01
Enter rolling type (year/month/week/day): week
Enter rolling increment (example: 1): 1

Loading file...
Dataset start date : 2020-01-01
Detected companies : 50
Rolling begins from : 2020-04-01
Writing Sheet: Week1 | Start: 2020-04-01
Writing Sheet: Week2 | Start: 2020-04-08
Writing Sheet: Week3 | Start: 2020-04-15
Writing Sheet: Week4 | Start: 2020-04-22
Writing Sheet: Week5 | Start: 2020-04-29
Writing Sheet: Week6 | Start: 2020-05-06
Writing Sheet: Week7 | Start: 2020-05-13
Writing Sheet: Week8 | Start: 2020-05-20
Writing Sheet: Week9 | Start: 2020-05-27
Writing Sheet: Week10 | Start: 2020-06-03
Writing Sheet: Week11 | Start: 2020-06-

Writing Sheet: Week182 | Start: 2023-09-20
Writing Sheet: Week183 | Start: 2023-09-27
Writing Sheet: Week184 | Start: 2023-10-04
Writing Sheet: Week185 | Start: 2023-10-11
Writing Sheet: Week186 | Start: 2023-10-18
Writing Sheet: Week187 | Start: 2023-10-25
Writing Sheet: Week188 | Start: 2023-11-01
Writing Sheet: Week189 | Start: 2023-11-08
Writing Sheet: Week190 | Start: 2023-11-15
Writing Sheet: Week191 | Start: 2023-11-22
Writing Sheet: Week192 | Start: 2023-11-29
Writing Sheet: Week193 | Start: 2023-12-06
Writing Sheet: Week194 | Start: 2023-12-13
Writing Sheet: Week195 | Start: 2023-12-20
Writing Sheet: Week196 | Start: 2023-12-27
Writing Sheet: Week197 | Start: 2024-01-03
Writing Sheet: Week198 | Start: 2024-01-10
Writing Sheet: Week199 | Start: 2024-01-17
Writing Sheet: Week200 | Start: 2024-01-24
Writing Sheet: Week201 | Start: 2024-01-31
Writing Sheet: Week202 | Start: 2024-02-07
Writing Sheet: Week203 | Start: 2024-02-14
Writing Sheet: Week204 | Start: 2024-02-21
Writing She

### Step 2 : step-by-step description for the Code: 4Combo_TotalPrices

1) Prompt for inputs — ask user for: price Excel path, combo quantity Excel path, output folder, OBS_DAYS (observation window in days), ROLL_TYPE (day/week/month/year) and ROLL_VALUE (increment). Create the output folder if missing.

2) Load price data — read the price Excel into df_price. Treat the first column as the date column and the rest as company price columns. Parse the date column to datetime and sort by date.

3) Skip first 3 months — compute skip_date = first_date + 3 months and keep only rows after this date in df_filtered. Print the original start and post-skip date.

4) Load combo data — read the combo Excel into df_combo, strip header whitespace, and split the Combo string into C1..C4 columns.

5) Parse quantities — convert the Quantities string into numeric Q1..Q4 using split_qty, filling missing or invalid entries with 0.0 and ensuring exactly 4 values.

6) Pre-map company indices — create col_index mapping each company column name to its integer index and convert C1..C4 into a combo_indices integer matrix (use -1 for missing companies).

7) Build quantity matrix — create qty_matrix as a numeric (float) array containing the four quantities per combo.

8) Helper to shift date — shift_date(start, roll_type, value) advances a date by the requested rolling frequency (day, week, month, year) and increment value.

9) Initialize rolling loop — set current_start to the earliest date in df_filtered and start window_count = 1.

10) Define observation window — each iteration sets window_end = current_start + OBS_DAYS - 1 and selects rows between current_start and window_end into df_window. If empty, the loop breaks.

11) Prepare vectorized price matrix — convert df_window[company_cols] to a NumPy array df_prices_np for fast numeric operations.

12) Vectorized per-row combo pricing — for each date row (vectorized by iterating rows of df_prices_np):

* extract all combo prices at once using price_matrix = dp_row[combo_indices] (with -1 replaced by 0),

* compute total_prices = sum(price_matrix * qty_matrix, axis=1) to get total combo price for every combo in one go.

13) Collect results quickly — append rows [DATE, Combo, Price_Combo (list), Total_Price] for every combo and update the tqdm progress bar.

14) Save window CSV — after processing the window, convert results to df_result and save it as Window_{n}_{OBS_DAYS}Days.csv in the output folder. Print the saved filename.

15) Advance rolling window — update current_start = shift_date(current_start, ROLL_TYPE, ROLL_VALUE), increment window_count, and repeat until current_start exceeds the dataset end.

16) Finish — when done, print a final success message: all rolling CSV files generated.

In [2]:
# 4Combo_TotalPrices

import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
import os

# ======================================================
#               USER INPUTS 
# ======================================================

file_prices = input("\nEnter price file path: ").strip()
file_combos = input("Enter combo quantity file path: ").strip()
output_folder = input("Enter output folder path: ").strip()

while True:
    try:
        OBS_DAYS = int(input("\nEnter observation days (example: 45): ").strip())
        break
    except:
        print("Invalid input! Enter a number.")

# ---------------- Rolling Frequency Input ----------------
valid_roll_types = ["day", "week", "month", "year"]

while True:
    ROLL_TYPE = input("\nEnter rolling type (day/week/month/year): ").strip().lower()
    if ROLL_TYPE in valid_roll_types:
        break
    print("Invalid! Choose from day / week / month / year")

while True:
    try:
        ROLL_VALUE = int(input("Enter rolling increment : ").strip())
        break
    except:
        print("Invalid number! Try again.")

# Make output folder
os.makedirs(output_folder, exist_ok=True)

# ======================================================
#                LOAD PRICE DATA
# ======================================================

df_price = pd.read_excel(file_prices)
date_col = df_price.columns[0]
company_cols = df_price.columns[1:]

df_price[date_col] = pd.to_datetime(df_price[date_col], dayfirst=True)
df_price = df_price.sort_values(date_col)

# Skip 3 months
first_date = df_price[date_col].min()
skip_date = first_date + pd.DateOffset(months=3)
df_filtered = df_price[df_price[date_col] > skip_date].copy()

print("\nDataset start:", first_date.date())
print("After skipping first 3 months:", skip_date.date())

# ======================================================
#                LOAD COMBO DATA
# ======================================================

df_combo = pd.read_excel(file_combos)
df_combo.columns = df_combo.columns.str.strip()

df_combo[['C1','C2','C3','C4']] = df_combo['Combo'].str.split(',', expand=True)

def split_qty(q):
    arr = []
    for x in str(q).split(","):
        try:
            arr.append(float(x.strip()))
        except:
            arr.append(0.0)
    # Ensure 4 elements
    while len(arr) < 4:
        arr.append(0.0)
    return arr[:4]

df_combo[['Q1','Q2','Q3','Q4']] = df_combo['Quantities'].apply(split_qty).tolist()

# Pre-map company index
col_index = {col: i for i, col in enumerate(company_cols)}

combo_indices = df_combo[['C1','C2','C3','C4']].applymap(
    lambda x: col_index.get(x, -1)
).values.astype(int)

qty_matrix = df_combo[['Q1','Q2','Q3','Q4']].values.astype(float)

# ======================================================
#         HELPER FUNCTION FOR ROLLING DATE OFFSET
# ======================================================

def shift_date(start, roll_type, value):
    """Return shifted date by selected rolling frequency."""
    if roll_type == "day":
        return start + timedelta(days=value)
    elif roll_type == "week":
        return start + timedelta(weeks=value)
    elif roll_type == "month":
        return start + pd.DateOffset(months=value)
    elif roll_type == "year":
        return start + pd.DateOffset(years=value)

# ======================================================
#               MAIN ROLLING WINDOW LOOP
# ======================================================

current_start = df_filtered[date_col].min()
window_count = 1

while current_start <= df_filtered[date_col].max():

    window_end = current_start + timedelta(days=OBS_DAYS - 1)

    df_window = df_filtered[
        (df_filtered[date_col] >= current_start) &
        (df_filtered[date_col] <= window_end)
    ].copy()

    if df_window.empty:
        break

    print(f"\nProcessing Window {window_count}")
    print("Start:", df_window[date_col].min().date())
    print("End  :", df_window[date_col].max().date())

    # ---------- Fast Calculation Section ----------
    results = []
    df_prices_np = df_window[company_cols].to_numpy(float)

    total_rows = len(df_window) * len(df_combo)
    pbar = tqdm(total=total_rows, desc=f"Window {window_count}", unit="rows")

    for idx, dp_row in enumerate(df_prices_np):
        date_value = df_window[date_col].iloc[idx]

        # Multiply entire combo price matrix at once
        price_matrix = dp_row[combo_indices]  
        price_matrix = np.where(combo_indices == -1, 0, price_matrix)

        total_prices = np.sum(price_matrix * qty_matrix, axis=1)

        # Create output rows fast
        for i in range(len(df_combo)):
            results.append([
                date_value,
                df_combo["Combo"].iloc[i],
                list(price_matrix[i]),
                total_prices[i]
            ])
            pbar.update(1)

    pbar.close()

    df_result = pd.DataFrame(results, columns=[
        "DATE", "Combo", "Price_Combo", "Total_Price"
    ])

    output_csv = os.path.join(
        output_folder,
        f"Window_{window_count}_{OBS_DAYS}Days.csv"
    )
    df_result.to_csv(output_csv, index=False)
    print("Saved:", output_csv)

    # ---------- Shift by User-defined Rolling Type ----------
    current_start = shift_date(current_start, ROLL_TYPE, ROLL_VALUE)
    window_count += 1

print("\nAll rolling CSV files generated successfully!")



Enter price file path: D:\Trading Stratergies\Stratergy 2 All Codes\50 comp data (input file).xlsx
Enter combo quantity file path: D:\Trading Stratergies\Backtesting Weekly\4company_combo_start_day_qty.xlsx
Enter output folder path: D:\Trading Stratergies\Backtesting Weekly\4company_combo_TotalPrice

Enter observation days (example: 45): 45

Enter rolling type (day/week/month/year): week
Enter rolling increment : 1

Dataset start: 2020-01-01
After skipping first 3 months: 2020-04-01


C:\Users\Swarupa\AppData\Local\Temp\3\ipykernel_7892\2087139177.py:88: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  combo_indices = df_combo[['C1','C2','C3','C4']].applymap(



Processing Window 1
Start: 2020-04-03
End  : 2020-05-15



Run 1:  10%|█████▌                                                     | 722474/7599900 [9:12:46<87:41:57, 21.78rows/s]

Window 1:  16%|█████████▏                                                | 986635/6218100 [00:15<01:10, 73919.80rows/s]


Window 1:  32%|██████████████████                                       | 1977203/6218100 [00:29<00:57, 74343.89rows/s]


...

Window 152: 100%|███████████████████████████████████████████████████████| 6218100/6218100 [01:36<00:00, 64634.92rows/s]


Saved: D:\Trading Stratergies\Backtesting Weekly\4company_combo_TotalPrice\Window_152_45Days.csv

Processing Window 153
Start: 2023-03-03
End  : 2023-04-13


Window 153: 100%|███████████████████████████████████████████████████████| 5987800/5987800 [01:36<00:00, 61799.56rows/s]


Saved: D:\Trading Stratergies\Backtesting Weekly\4company_combo_TotalPrice\Window_153_45Days.csv

Processing Window 154
Start: 2023-03-10
End  : 2023-04-21


Window 154:   9%|█████                                                   | 562228/6218100 [00:07<01:14, 75581.84rows/s]

### Step 3: step-by-step description for the Code: 4Company_EURINR_Ratio_50DMA
1) Prompt user for inputs — ask for the input folder containing price CSVs, the EURINR Excel file path, output folder, and the moving-average window MOVING_AVG_DAYS. Create the output folder if missing.

2) Load FX (EURINR) series — read the EURINR Excel (DATE, EURINR), convert DATE to datetime, and set it as the index for fast joins.

3) Discover input CSV files — find and sort all *.csv files in the input price folder and print the count.

4) Loop over each input CSV file — for every file create an output filename prefixed with the DMA window (e.g., 50DMA_...).

5) Prepare rolling state per combo — create two structures:

* ratio_queue: a defaultdict of deque(maxlen=MOVING_AVG_DAYS) to hold the last N ratio values per combo.

* ratio_sum: a defaultdict(float) to keep the running sum per combo (so you can compute the average quickly).

6) Stream the CSV in chunks — read the large CSV file with pd.read_csv(..., chunksize=50000) selecting only DATE, Combo, Total_Price and parsing DATE. A progress bar (tqdm) shows progress.

7) Join FX into each chunk — for each chunk, join the fx table on DATE to attach the matching EURINR value; drop rows where EURINR is missing.

8) Compute EURINR ratio per row — calculate eur_ratio = Total_Price / EURINR as a NumPy array for the chunk.

9) Compute rolling DMA per combo — iterate rows in the chunk:

* For each row’s Combo, maintain its deque and ratio_sum.

* If the deque is full, subtract the oldest value from ratio_sum.

* Append the new ratio, add it to ratio_sum.

* Compute DMA = ratio_sum[combo] / len(deque) and store it.

10) Assemble output chunk — build a DataFrame with DATE, Combo, EURINR_Ratio, and the computed {MOVING_AVG_DAYS}DMA_EURINR.

11) Append to output CSV — write the chunk to the output file in append mode; write header only for the first chunk.

12) Repeat until file complete — loop through all chunks for the file. After finishing, print the created output path.

13) Process all files — repeat steps 4–12 for every input CSV discovered.

14) Finish — when all files are processed, print a final success message.

In [1]:
# 4Company_EURINR_Ratio_50DMA

import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import os
from collections import deque, defaultdict

# ====================== REQUIRED USER INPUTS ======================
INPUT_PRICE_FOLDER = input("Enter input PRICE folder path: ").strip()
INPUT_FX_FILE = input("Enter FX (EURINR) file path: ").strip()
OUTPUT_FOLDER = input("Enter OUTPUT folder path: ").strip()
MOVING_AVG_DAYS = int(input("Enter Moving Average days : "))

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== LOAD FX DATA ======================
fx = pd.read_excel(INPUT_FX_FILE, usecols=["DATE", "EURINR"])
fx["DATE"] = pd.to_datetime(fx["DATE"])
fx.set_index("DATE", inplace=True)

# ====================== GET ALL INPUT FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_PRICE_FOLDER, "*.csv")))
print(f"Total input CSV files detected: {len(csv_files)}")

# ====================== PROCESS EACH FILE ======================
for file_csv in csv_files:

    print(f"\nProcessing: {os.path.basename(file_csv)}")

    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"{MOVING_AVG_DAYS}DMA_{os.path.basename(file_csv)}"
    )

    # Deque and cumulative sum per Combo
    ratio_queue = defaultdict(lambda: deque(maxlen=MOVING_AVG_DAYS))
    ratio_sum = defaultdict(float)

    header_written = False

    for chunk in tqdm(
        pd.read_csv(
            file_csv,
            chunksize=50000,
            parse_dates=["DATE"],
            usecols=["DATE", "Combo", "Total_Price"]
        ),
        desc="Streaming",
        unit=" rows"
    ):
        # Merge EURINR data
        chunk = chunk.join(fx, on="DATE", how="left")
        chunk.dropna(subset=["EURINR"], inplace=True)

        if chunk.empty:
            continue

        # Compute EURINR Ratio
        eur_ratio = chunk["Total_Price"].values / chunk["EURINR"].values
        dma_values = np.zeros(len(chunk))
        combos = chunk["Combo"].values

        # Rolling moving average calculation
        for i, combo in enumerate(combos):
            q = ratio_queue[combo]

            # Maintain rolling sum
            if len(q) == MOVING_AVG_DAYS:
                ratio_sum[combo] -= q[0]

            q.append(eur_ratio[i])
            ratio_sum[combo] += eur_ratio[i]

            dma_values[i] = ratio_sum[combo] / len(q)

        # Save chunk to output
        out = pd.DataFrame({
            "DATE": chunk["DATE"].values,
            "Combo": combos,
            "EURINR_Ratio": eur_ratio,
            f"{MOVING_AVG_DAYS}DMA_EURINR": dma_values
        })

        out.to_csv(output_file, mode="a", index=False, header=not header_written)
        header_written = True

    print(f"Output created: {output_file}")

print("\n ALL FILES PROCESSED SUCCESSFULLY")


Enter input PRICE folder path: C:\Users\Swarupa\Desktop\input\4company_combo_TotalPrice
Enter FX (EURINR) file path: C:\Users\Swarupa\Desktop\input\EURINR_Values(input file).xlsx
Enter OUTPUT folder path: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_EURINR_50DMA
Enter Moving Average days : 50
Total input CSV files detected: 202

Processing: Window_100_45Days.csv


Streaming: 134 rows [01:01,  2.17 rows/s]


Output created: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_EURINR_50DMA\50DMA_Window_100_45Days.csv

Processing: Window_101_45Days.csv


Streaming: 129 rows [00:59,  2.17 rows/s]


Output created: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_EURINR_50DMA\50DMA_Window_101_45Days.csv

Processing: Window_102_45Days.csv


...

Output created: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_EURINR_50DMA\50DMA_Window_98_45Days.csv

Processing: Window_99_45Days.csv


Streaming: 134 rows [01:04,  2.08 rows/s]


Output created: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_EURINR_50DMA\50DMA_Window_99_45Days.csv

Processing: Window_9_45Days.csv


Streaming: 143 rows [01:09,  2.07 rows/s]

Output created: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_EURINR_50DMA\50DMA_Window_9_45Days.csv

 ALL FILES PROCESSED SUCCESSFULLY


### Step 4: step-by-step description for the Code:  4Company_NIFTY_Ratio_50DMA
1) Prompt for inputs — ask user for: input price folder, NIFTY Excel file, output folder, and MOVING_AVG_DAYS. Create output folder if missing.

2) Load NIFTY series — read DATE and NIFTY from the Excel file, convert DATE to datetime, and set it as the DataFrame index for fast joins.

3) Find input CSVs — list and sort all *.csv files in the input price folder and print the count.

4) Loop over each CSV file — for each price CSV, build an output filename prefixed with the DMA window (e.g., 50DMA_NIFTY_filename.csv).

5) Prepare rolling state per combo — create:

* ratio_queue: defaultdict of deque(maxlen=MOVING_AVG_DAYS) to store last N ratio values per combo, and

* ratio_sum: defaultdict(float) to hold running sums for fast average computation.

6) Stream the CSV in chunks — read the CSV in chunks (chunksize=50000) selecting DATE, Combo, Total_Investment. A tqdm progress bar shows progress.

7) Join NIFTY values into chunk — join the nifty table on DATE (left join) and drop rows without a matching NIFTY.

8) Compute per-row ratio — compute nifty_ratio = Total_Investment / NIFTY for the chunk as a NumPy array.

9) Initialize DMA array — create a zero dma_values array to store the moving average for each row in the chunk.

10) Per-row rolling average update — for each row’s Combo:

* fetch its deque q;

* if q is full, subtract the oldest value from ratio_sum[combo];

* append the new ratio to q and add it to ratio_sum[combo];

* compute dma_values[i] = ratio_sum[combo] / len(q).

11) Assemble output chunk — build a DataFrame with DATE, Combo, NIFTY_Ratio, and {MOVING_AVG_DAYS}DMA_NIFTY.

12) Append to output CSV — write the output chunk to the CSV in append mode; only write the header for the first chunk (header_written flag).

13) Repeat until file complete — continue processing chunks until the CSV is exhausted, then print the saved output path.

14) Process all files — repeat steps 4–13 for every input CSV file found.

15) Finish — when all files are done print the final success message.

In [2]:
# 4Company_NIFTY_Ratio_50DMA

import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import os
from collections import deque, defaultdict

# ====================== REQUIRED USER INPUTS ======================
INPUT_PRICE_FOLDER = input("Enter input PRICE folder path: ").strip()
INPUT_NIFTY_FILE = input("Enter NIFTY file path: ").strip()
OUTPUT_FOLDER = input("Enter OUTPUT folder path: ").strip()
MOVING_AVG_DAYS = int(input("Enter Moving Average days: "))

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== LOAD NIFTY DATA ======================
nifty = pd.read_excel(INPUT_NIFTY_FILE, usecols=["DATE", "NIFTY"])
nifty["DATE"] = pd.to_datetime(nifty["DATE"])
nifty.set_index("DATE", inplace=True)

# ====================== GET ALL INPUT FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_PRICE_FOLDER, "*.csv")))
print(f"Total input CSV files detected: {len(csv_files)}")

# ====================== PROCESS EACH FILE ======================
for file_csv in csv_files:

    print(f"\nProcessing: {os.path.basename(file_csv)}")

    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"{MOVING_AVG_DAYS}DMA_NIFTY_{os.path.basename(file_csv)}"
    )

    # Rolling queue + sum per Combo
    ratio_queue = defaultdict(lambda: deque(maxlen=MOVING_AVG_DAYS))
    ratio_sum = defaultdict(float)

    header_written = False

    # Stream CSV in chunks
    for chunk in tqdm(
        pd.read_csv(
            file_csv,
            chunksize=50000,
            parse_dates=["DATE"],
            usecols=["DATE", "Combo", "Total_Price"]
        ),
        desc="Streaming",
        unit=" rows"
    ):
        # Merge NIFTY data
        chunk = chunk.join(nifty, on="DATE", how="left")
        chunk.dropna(subset=["NIFTY"], inplace=True)

        if chunk.empty:
            continue

        # Compute Ratio
        nifty_ratio = chunk["Total_Price"].values / chunk["NIFTY"].values
        dma_values = np.zeros(len(chunk))

        combos = chunk["Combo"].values

        # Rolling Moving Average Calculation
        for i, combo in enumerate(combos):
            q = ratio_queue[combo]

            # Maintain rolling sum
            if len(q) == MOVING_AVG_DAYS:
                ratio_sum[combo] -= q[0]

            q.append(nifty_ratio[i])
            ratio_sum[combo] += nifty_ratio[i]

            dma_values[i] = ratio_sum[combo] / len(q)

        # Prepare output chunk
        out = pd.DataFrame({
            "DATE": chunk["DATE"].values,
            "Combo": combos,
            "NIFTY_Ratio": nifty_ratio,
            f"{MOVING_AVG_DAYS}DMA_NIFTY": dma_values
        })

        out.to_csv(output_file, mode="a", index=False, header=not header_written)
        header_written = True

    print(f"Output saved: {output_file}")

print("\n ALL FILES PROCESSED SUCCESSFULLY.")


Enter input PRICE folder path: C:\Users\Swarupa\Desktop\input\4company_combo_TotalPrice
Enter NIFTY file path: C:\Users\Swarupa\Desktop\input\Nifty_Index_values(input file).xlsx
Enter OUTPUT folder path: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_NIFTY_50DMA
Enter Moving Average days: 50
Total input CSV files detected: 202

Processing: Window_100_45Days.csv


Streaming: 134 rows [01:04,  2.09 rows/s]


Output saved: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_NIFTY_50DMA\50DMA_NIFTY_Window_100_45Days.csv

Processing: Window_101_45Days.csv


Streaming: 129 rows [01:02,  2.07 rows/s]


Output saved: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_NIFTY_50DMA\50DMA_NIFTY_Window_101_45Days.csv

Processing: Window_102_45Days.csv


...

Streaming: 129 rows [01:04,  2.00 rows/s]


Output saved: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_NIFTY_50DMA\50DMA_NIFTY_Window_80_45Days.csv

Processing: Window_81_45Days.csv


Streaming: 129 rows [01:02,  2.05 rows/s]


Output saved: D:\Trading Stratergies\Backtesting weekly\4Company_Combo_NIFTY_50DMA\50DMA_NIFTY_Window_81_45Days.csv

Processing: Window_82_45Days.csv


Streaming: 41 rows [00:21,  2.13 rows/s]

### Step 5: step-by-step description for the Code: EURINR_Ratio > 50DMA_EURINR
1) Prompt for inputs — ask the user for the folder containing EURINR DMA CSV files and an output folder for filtered results; create the output folder if it doesn't exist.

2) Find CSV files — list and sort all *.csv files in the input folder and print how many were found.

3) Loop over each CSV — process files one by one and print the current filename.

4) Read file into DataFrame — load the CSV with pd.read_csv().

5) Auto-detect DMA column — find the first column whose name contains "DMA_EURINR" (e.g., 50DMA_EURINR) and store its name in dma_col.

6) Apply filter condition — keep only rows where EURINR_Ratio > df[dma_col].

7) Skip empty results — if no rows satisfy the condition, print a message and move to the next file.

8) Build output filename — create a new filename prefixed with Filtered_ inside the output folder.

9) Save filtered results — write the filtered DataFrame to CSV (no index) and print the saved path.

10) Repeat for all files — process every detected CSV.

11) Finish — print a final success message when all files are handled.

In [4]:
#EURINR_Ratio > 50DMA_EURINR

import pandas as pd
import glob
import os

# ====================== USER INPUTS ======================
INPUT_FOLDER = input("Enter folder path of EURINR DMA CSV files: ").strip()
OUTPUT_FOLDER = input("Enter output folder path for filtered results: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== PROCESS FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"Total CSV files detected: {len(csv_files)}")

for file in csv_files:

    file_name = os.path.basename(file)
    print(f"\nProcessing: {file_name}")

    # Load file
    df = pd.read_csv(file)

    # Detect DMA column automatically (example: "50DMA_EURINR")
    dma_col = [col for col in df.columns if "DMA_EURINR" in col][0]

    # ================= APPLY CONDITION =================
    df_filtered = df[df["EURINR_Ratio"] > df[dma_col]].copy()

    # Count filtered rows
    filtered_count = len(df_filtered)

    print(f" Combinations satisfying condition: {filtered_count}")

    if df_filtered.empty:
        print("  No combinations satisfy condition → Skipped saving.")
        continue

    # Save file
    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"Filtered_{file_name}"
    )

    df_filtered.to_csv(output_file, index=False)
    print(f" Saved filtered file: {output_file}")

print("\nALL FILES FILTERED SUCCESSFULLY.")


Enter folder path of EURINR DMA CSV files: E:\Backtesting weekly\4Company_Combo_EURINR_50DMA
Enter output folder path for filtered results:  E:\Backtesting weekly\EURINR_ratio_GT_50DMA


### Step 6: step-by-step description for the Code: NIFTY_Ratio > 50DMA_NIFTY
1) Prompt for inputs — ask the user for the folder containing NIFTY DMA CSV files and the output folder for filtered results; create the output folder if needed.

2) Find all CSV files — list and sort every *.csv file in the input folder and print the total detected.

3) Process each CSV file one by one — print the filename being processed.

4) Load the CSV — read the file using pd.read_csv().

5) Auto-detect the NIFTY DMA column — search for the first column name that contains "DMA_NIFTY" (e.g., "50DMA_NIFTY"), and store that column name.

6) Apply the filter condition — keep only rows where: NIFTY_Ratio > DMA_NIFTY

7) That is, combos whose ratio is greater than their moving-average value.

8) Check for empty results — if no rows meet the condition, print a message and skip saving.

9) Create output filename — prefix the filename with "Filtered_" and save it in the output folder.

10) Save filtered rows — write the filtered DataFrame to CSV (without index) and print the saved path.

11) Repeat for all files — loop through every NIFTY DMA file.

12) Finish — show a success message once all processing is complete.

In [6]:
#NIFTY_Ratio > 50DMA_NIFTY

import pandas as pd
import glob
import os

# ====================== USER INPUTS ======================
INPUT_FOLDER = input("Enter folder path of NIFTY DMA CSV files: ").strip()
OUTPUT_FOLDER = input("Enter output folder path for filtered results: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ====================== PROCESS FILES ======================
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"Total CSV files detected: {len(csv_files)}")

for file in csv_files:

    file_name = os.path.basename(file)
    print(f"\nProcessing: {file_name}")

    # Load file
    df = pd.read_csv(file)

    # Detect DMA column automatically (example: 50DMA_NIFTY)
    dma_col = [col for col in df.columns if "DMA_NIFTY" in col][0]

    # ================= APPLY CONDITION =================
    df_filtered = df[df["NIFTY_Ratio"] > df[dma_col]].copy()

    # Count filtered rows
    filtered_count = len(df_filtered)

    print(f" Combinations satisfying condition: {filtered_count}")

    if df_filtered.empty:
        print("  No combinations satisfy condition → Skipped saving.")
        continue

    # Save file
    output_file = os.path.join(
        OUTPUT_FOLDER,
        f"Filtered_{file_name}"
    )

    df_filtered.to_csv(output_file, index=False)
    print(f" Saved filtered file: {output_file}")

print("\nALL FILES FILTERED SUCCESSFULLY.")


Enter folder path of NIFTY DMA CSV files: E:\Backtesting weekly\4Company_Combo_NIFTY_50DMA
Enter output folder path for filtered results:  E:\Backtesting weekly\NIFTY_Ratio_50DMA


### Step 7: step-by-step description for the Code: common_combinations_EURINR_NIFTY
1) Prompt for input folders — ask user for: EURINR CSV folder, NIFTY CSV folder, and output folder. Create the output folder if it doesn’t exist.

2) List and sort files — get all *.csv files from EURINR and NIFTY folders; print counts of detected files.

3) Helper function extract_month — extract the month identifier (e.g., Month_1_45Days) from the filename to match corresponding EURINR and NIFTY files.

4) Loop through EURINR files — for each EURINR CSV file:

* Extract its month identifier using extract_month.

* Find the matching NIFTY file with the same month; skip if not found.

5) Load CSVs — read both EURINR and matching NIFTY files into DataFrames.

6) Convert DATE to datetime — for both DataFrames to enable proper merging and filtering.

7) Identify common combos — compute the intersection of Combo values in EURINR and NIFTY for the same month; print the counts.

8) Skip months with no common combos — if none, continue to the next EURINR file.

9) Filter to common combos — keep only rows with combos that exist in both EURINR and NIFTY files.

10) Merge datasets — merge the filtered EURINR and NIFTY DataFrames on DATE and Combo, adding suffixes _EURINR and _NIFTY for columns from each source.

11) Save merged CSV — write the merged DataFrame to the output folder with filename like Common_Month_1_45Days.csv.

12) Repeat for all EURINR files — process each month sequentially.

13) Finish — print a success message when all months are processed.

In [7]:
#common_combinations_EURINR_NIFTY

import pandas as pd
import glob
import os

# ====================== USER INPUTS ==========================
EURINR_FOLDER = input("Enter folder path of EURINR files: ").strip()
NIFTY_FOLDER  = input("Enter folder path of NIFTY files: ").strip()
OUTPUT_FOLDER = input("Enter output folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

eur_files = sorted(glob.glob(os.path.join(EURINR_FOLDER, "*.csv")))
nifty_files = sorted(glob.glob(os.path.join(NIFTY_FOLDER, "*.csv")))

print(f"EURINR files found : {len(eur_files)}")
print(f"NIFTY  files found : {len(nifty_files)}")

def extract_month(file_name):
    """
    Example filename:
        50DMA_EURINR_Month_1_45Days.csv
        50DMA_NIFTY_Month_1_45Days.csv
    Function extracts: "Month_1_45Days"
    """
    parts = file_name.split("_")
    return "_".join(parts[-2:]).replace(".csv", "")   # Month_x_45Days

# Loop through all EURINR files
for eur_file in eur_files:

    eur_name = os.path.basename(eur_file)
    eur_month = extract_month(eur_name)

    # Find matching NIFTY file for SAME MONTH
    matching_nifty = [f for f in nifty_files if extract_month(os.path.basename(f)) == eur_month]

    if not matching_nifty:
        print(f"\n No NIFTY file found for: {eur_name}")
        continue

    nifty_file = matching_nifty[0]

    print(f"\n==========================")
    print(f" Processing Month: {eur_month}")
    print(f" EURINR → {eur_name}")
    print(f" NIFTY  → {os.path.basename(nifty_file)}")
    print(f"==========================")

    # Load files
    df_eur = pd.read_csv(eur_file)
    df_nifty = pd.read_csv(nifty_file)

    # Convert DATE to datetime
    df_eur["DATE"] = pd.to_datetime(df_eur["DATE"])
    df_nifty["DATE"] = pd.to_datetime(df_nifty["DATE"])

    # Select common combos within SAME MONTH files
    common_combos = sorted(set(df_eur["Combo"]).intersection(df_nifty["Combo"]))

    print(f" EURINR combos: {df_eur['Combo'].nunique()}")
    print(f" NIFTY combos : {df_nifty['Combo'].nunique()}")
    print(f" Common combos: {len(common_combos)}")

    if len(common_combos) == 0:
        print(" No common combinations → Skipping this month.")
        continue

    # Filter EURINR and NIFTY to only common combos
    df_eur_f = df_eur[df_eur["Combo"].isin(common_combos)]
    df_nifty_f = df_nifty[df_nifty["Combo"].isin(common_combos)]

    # Merge on DATE + Combo → ONE DATE column only
    df_final = df_eur_f.merge(
        df_nifty_f,
        on=["DATE", "Combo"],
        suffixes=("_EURINR", "_NIFTY")
    )

    # Save monthly output
    output_path = os.path.join(OUTPUT_FOLDER, f"Common_{eur_month}.csv")
    df_final.to_csv(output_path, index=False)

    print(f" Saved: {output_path}")

print("\n ALL MONTHS PROCESSED SUCCESSFULLY!")


Enter folder path of EURINR files: E:\Backtesting weekly\EURINR_ratio_GT_50DMA
Enter folder path of NIFTY files: E:\Backtesting weekly\NIFTY_Ratio_GT_50DMA
Enter output folder path:  E:\Backtesting weekly\common_combinations_EURINR_NIFTY


### Step 8: step-by-step description for the Code: Ratio <= 1.03*50DMA

1) Prompt for inputs — ask the user for the folder containing CSV files and an output folder; create the output folder if it doesn't exist.

2) List CSV files — find and sort all *.csv files in the input folder and print how many were found.

3) Prepare output filename — for each input CSV, build an output path in the output folder using the same base filename.

4) Read CSV — load the current CSV with DATE parsed as datetime.

5) Check required columns — verify the file contains all these columns:
["DATE","Combo","EURINR_Ratio","50DMA_EURINR","NIFTY_Ratio","50DMA_NIFTY"].
* If any are missing, print which ones and skip the file.

6) Apply filter conditions — keep only rows that satisfy both:

* EURINR_Ratio <= 1.03 * 50DMA_EURINR

* NIFTY_Ratio <= 1.03 * 50DMA_NIFTY

7) Skip empty results — if no rows meet the conditions, print a message and skip saving.

8) Save filtered CSV — write the filtered DataFrame to the output folder using the same filename (no index), and print the saved path and row count.

9) Repeat for all files — process every detected CSV file in the input folder.

10) Finish — print a final message when processing is done.

In [8]:
#Ratio <= 1.03*50DMA

import os
import glob
import pandas as pd

# ================= USER INPUTS =================
INPUT_FOLDER  = input("Enter folder path with CSVs: ").strip()
OUTPUT_FOLDER = input("Enter output folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Helper to create output CSV filename
def get_output_csv_name(input_csv):
    base = os.path.basename(input_csv)
    return os.path.join(OUTPUT_FOLDER, base)

# Load all CSV files
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"\nTotal CSV files detected: {len(csv_files)}")

for csv_file in csv_files:
    output_csv = get_output_csv_name(csv_file)
    print(f"\nProcessing {os.path.basename(csv_file)} ...")

    df = pd.read_csv(csv_file, parse_dates=["DATE"])

    required_cols = ["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR", "NIFTY_Ratio", "50DMA_NIFTY"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f" Missing columns: {missing}. Skipping file.")
        continue

    # Keep only combinations satisfying both conditions
    df_filtered = df[
        (df["EURINR_Ratio"] <= 1.03 * df["50DMA_EURINR"]) &
        (df["NIFTY_Ratio"] <= 1.03 * df["50DMA_NIFTY"])
    ]

    # Skip file if no rows satisfy the condition
    if df_filtered.empty:
        print("  No rows satisfy the condition. File skipped.")
        continue

    # Save filtered CSV
    df_filtered.to_csv(output_csv, index=False)
    print(f"  Filtered CSV saved: {output_csv} (rows: {len(df_filtered)})")

print("\nDONE! Only combinations that satisfy the condition are saved.")


Enter folder path with CSVs: E:\Backtesting weekly\common_combinations_EURINR_NIFTY
Enter output folder path: E:\Backtesting weekly\Ratio_should_not_be_GT_50DMA


### Step 9: step-by-step description for the Code: Ratio > 1.01*50DMA

1) Prompt for inputs — ask for the folder containing CSV files and an output folder; create the output folder if it doesn't exist.

2) List CSV files — find and sort all *.csv files in the input folder and print the total found.

3) Initialize counter — total_combinations = 0 to accumulate matches across all files.

4) Loop over each CSV — for each file build an output path (same base name, in output folder) and print the file being processed.

5) Read CSV — load the file with DATE parsed as datetime.

6) Verify required columns — check for ["DATE","Combo","EURINR_Ratio","50DMA_EURINR","NIFTY_Ratio","50DMA_NIFTY"]; if any are missing, report and skip the file.

7) Apply filter condition — keep only rows where both:

* EURINR_Ratio > 1.01 * 50DMA_EURINR and

* NIFTY_Ratio > 1.01 * 50DMA_NIFTY.

8) Count & accumulate — compute num_combinations = len(df_filtered) and add to total_combinations. If zero, skip saving.

9) Save filtered results — write the filtered DataFrame to the output CSV (no index) and print how many rows satisfied the condition.

10) Repeat for all files — continue until every CSV is processed.

11) Finish — print final summary showing total_combinations across all processed CSVs.

In [9]:
#Ratio > 1.01*50DMA

import os
import glob
import pandas as pd

# ================= USER INPUTS =================
INPUT_FOLDER  = input("Enter folder path with CSVs: ").strip()
OUTPUT_FOLDER = input("Enter output folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Helper to create output CSV filename
def get_output_csv_name(input_csv):
    base = os.path.basename(input_csv)
    return os.path.join(OUTPUT_FOLDER, base)

# Load all CSV files
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"\nTotal CSV files detected: {len(csv_files)}")

total_combinations = 0  

for csv_file in csv_files:
    output_csv = get_output_csv_name(csv_file)
    print(f"\nProcessing {os.path.basename(csv_file)} ...")

    df = pd.read_csv(csv_file, parse_dates=["DATE"])

    required_cols = ["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR", "NIFTY_Ratio", "50DMA_NIFTY"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        print(f" Missing columns: {missing}. Skipping file.")
        continue

    # Keep only combinations satisfying Ratio > 1.01 * 50DMA
    df_filtered = df[
        (df["EURINR_Ratio"] > 1.01 * df["50DMA_EURINR"]) &
        (df["NIFTY_Ratio"] > 1.01 * df["50DMA_NIFTY"])
    ]

    num_combinations = len(df_filtered)
    total_combinations += num_combinations

    if num_combinations == 0:
        print("  No rows satisfy the condition. File skipped.")
        continue

    # Save filtered CSV
    df_filtered.to_csv(output_csv, index=False)
    print(f"  Filtered CSV saved: {output_csv} (rows satisfying condition: {num_combinations})")

print("\nDONE!")
print(f"Total combinations across all CSVs that satisfy the condition: {total_combinations}")


Enter folder path with CSVs: E:\Backtesting weekly\Ratio_should_not_be_GT_50DMA
Enter output folder path: E:\Backtesting weekly\Ratio_GT_50DMA


### Step 10: step-by-step description for the Code: Top 25% combinations by Average_Ratio

1) Prompt for inputs — ask the user for: folder containing filtered CSVs (INPUT_FOLDER), an output Excel file path (OUTPUT_FILE), and the top percentage to select (TOP_PERCENT). Validate TOP_PERCENT is between 1 and 100.

2) Prepare output folder — create the parent folder for OUTPUT_FILE if it doesn't already exist.

3) Helper for sheet names — define get_sheet_name() to convert each CSV filename into a safe Excel sheet name (max 31 characters).

4) Find filtered CSVs — list and sort all *.csv files in INPUT_FOLDER and print the count.

5) Open Excel writer — start an openpyxl ExcelWriter to collect all top-percent results into one workbook.

6) Loop over each CSV file — read the CSV (parse DATE) and print the filename being processed.

7) Verify required columns — confirm the file contains ["EURINR_Ratio","NIFTY_Ratio","Combo","DATE"]. If any are missing, skip the file and report which columns are absent.

8) Compute internal score — create a temporary _Score column equal to EURINR_Ratio + NIFTY_Ratio to rank combinations.

9) Determine top count — compute top_count = max(1, int((TOP_PERCENT / 100) * len(df))) so at least one row is selected.

10) Select top rows — use nlargest(top_count, "_Score") to pick the highest-scoring combinations.

11) Clean up — drop the temporary _Score column before saving.

12) Write to Excel sheet — save the top-percent rows to a sheet named after the CSV (truncated to 31 chars) inside the output workbook; print the sheet name and row count.

13) Repeat for all files — process each CSV the same way and add a sheet per file.

14) Finish — close the Excel writer (file saved) and print the final message with the output file path.

In [10]:
#Top 25% combinations by Average_Ratio

import os
import glob
import pandas as pd

# ================= USER INPUTS =================
INPUT_FOLDER  = input("Enter folder path with filtered CSVs: ").strip()
OUTPUT_FILE   = input("Enter output Excel file path: ").strip()
TOP_PERCENT   = float(input("Enter Top Percentage (example: 25 for top 25%): ").strip())

# Ensure valid % value
if TOP_PERCENT <= 0 or TOP_PERCENT > 100:
    print("Invalid percentage! Please enter value between 1 and 100.")
    exit()

# Ensure output folder exists
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

# Create safe Excel sheet names
def get_sheet_name(csv_file):
    name = os.path.basename(csv_file).replace(".csv", "")
    return name[:31]  

# Load all filtered CSV files
csv_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.csv")))
print(f"\nTotal filtered CSV files detected: {len(csv_files)}")

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:

    for csv_file in csv_files:
        print(f"\nProcessing {os.path.basename(csv_file)} ...")
        df = pd.read_csv(csv_file, parse_dates=["DATE"])

        required_cols = ["EURINR_Ratio", "NIFTY_Ratio", "Combo", "DATE"]
        missing = [c for c in required_cols if c not in df.columns]
        if missing:
            print(f" Missing columns {missing}. Skipping file.")
            continue

        # Internal ranking score 
        df["_Score"] = df["EURINR_Ratio"] + df["NIFTY_Ratio"]

        # Calculate count of top X%
        top_count = max(1, int((TOP_PERCENT / 100) * len(df)))

        # Select top rows
        df_top = df.nlargest(top_count, "_Score")

        # Remove internal column before saving
        df_top = df_top.drop(columns=["_Score"])

        # Write sheet
        sheet_name = get_sheet_name(csv_file)
        df_top.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"  Saved Top {TOP_PERCENT}% → Sheet: {sheet_name} (Rows: {len(df_top)})")

print("\nDONE!")
print(f"All Top {TOP_PERCENT}% combinations saved to:", OUTPUT_FILE)


Enter folder path with filtered CSVs: E:\Backtesting weekly\Ratio_GT_50DMA
Enter output Excel file path: E:\Backtesting weekly\Top_25%_combinations_by Average_Ratio.xlsx
Enter Top Percentage (example: 25 for top 25%): 25


### Step 11: step-by-step description for the Code: Lowest_Average_Correlation

1) Prompt for file paths — ask the user for the correlation Excel (file_corr), the multi-sheet data Excel (file_data), and the output Excel path (output_file). Create the output directory if needed.

2) Normalize combo function — normalize_combo(combo) splits a combo string by commas, strips whitespace, sorts stock names alphabetically, and rejoins them (so "B,A" and "A,B" become the same).

3) Load correlation file — read file_corr into df_corr and add a helper column Combo_Normalized by applying normalize_combo.

4) Open data workbook — load file_data with pd.ExcelFile() to access all sheet names.

5) Iterate sheets — loop through every sheet in the data workbook and read it into df_data.

6) Normalize combos in data — create df_data["Combo_Normalized"] using the same normalization function.

7) Merge correlation values — merge df_data with df_corr[["Combo_Normalized","Average_Realized_Correlation"]] on Combo_Normalized using a left join so each combo gets its Average_Realized_Correlation where available.

8) Drop unmatched rows — remove rows that did not find a correlation (dropna on Average_Realized_Correlation) so only combos with correlation remain.

9) Clean helper column — remove the Combo_Normalized helper column from the merged DataFrame.

10) Store per-sheet results — keep the resulting DataFrame for each sheet in the output_sheets dict.

11) Write output workbook — open an openpyxl ExcelWriter and write each processed sheet back to the output Excel file with the same sheet names.

12) Finish — close the writer (saving the file) and print a completion message: correlation values are applied and the multi-sheet output is saved.

In [1]:
#Top_25%_Lowest_Average_Correlation

import pandas as pd
import os

# ---------------- USER INPUT ----------------
file_corr = input("Enter the path of the Correlation Excel file: ").strip()
file_data = input("Enter the path of the data Excel file (multiple sheets): ").strip()
output_file = input("Enter the path for the output Excel file: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

def normalize_combo(combo):
    if pd.isna(combo):
        return combo
    stocks = [s.strip() for s in combo.split(",")]
    stocks_sorted = sorted(stocks)
    return ", ".join(stocks_sorted)

print("Loading correlation file...")
df_corr = pd.read_excel(file_corr)
df_corr["Combo_Normalized"] = df_corr["Combo"].apply(normalize_combo)

print("Loading data file with multiple sheets...")
xls_data = pd.ExcelFile(file_data)
sheet_names = xls_data.sheet_names

output_sheets = {}

for sheet in sheet_names:
    print(f"Processing sheet: {sheet} ...")
    df_data = pd.read_excel(xls_data, sheet_name=sheet)
    
    # Normalize combo
    df_data["Combo_Normalized"] = df_data["Combo"].apply(normalize_combo)
    
    # Merge correlation
    df_merged = df_data.merge(
        df_corr[["Combo_Normalized", "Average_Realized_Correlation"]],
        on="Combo_Normalized",
        how="left"
    )
    
    # Drop rows without correlation if needed
    df_merged = df_merged.dropna(subset=["Average_Realized_Correlation"])
    
    # Remove helper column
    df_merged.drop(columns=["Combo_Normalized"], inplace=True)
    
    output_sheets[sheet] = df_merged

print("Saving output file with multiple sheets...")
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet, df_out in output_sheets.items():
        df_out.to_excel(writer, sheet_name=sheet, index=False)

print("DONE: Correlation values applied and output saved with multiple sheets.")


Enter the path of the Correlation Excel file: D:\Trading Stratergies\Backtesting\4Stock_Combinations_Correlation.xlsx
Enter the path of the data Excel file (multiple sheets): E:\Backtesting weekly\Top_25%_combinations_by Average_Ratio.xlsx
Enter the path for the output Excel file: E:\Backtesting weekly\Top_25%_Lowest_Average_Correlation.xlsx
Loading correlation file...
Loading data file with multiple sheets...
Processing sheet: Common_100_45Days ...
Processing sheet: Common_101_45Days ...
Processing sheet: Common_102_45Days ...
Processing sheet: Common_103_45Days ...
Processing sheet: Common_104_45Days ...
Processing sheet: Common_105_45Days ...
Processing sheet: Common_106_45Days ...
Processing sheet: Common_107_45Days ...
Processing sheet: Common_108_45Days ...
Processing sheet: Common_109_45Days ...
Processing sheet: Common_10_45Days ...
Processing sheet: Common_110_45Days ...
Processing sheet: Common_111_45Days ...
Processing sheet: Common_112_45Days ...
Processing sheet: Common_11

Processing sheet: Common_97_45Days ...
Processing sheet: Common_98_45Days ...
Processing sheet: Common_99_45Days ...
Processing sheet: Common_9_45Days ...
Saving output file with multiple sheets...
DONE: Correlation values applied and output saved with multiple sheets.


### Step 12: step-by-step description for the Code: Correlation_with_Japan_Bond_Yield( for each Combo)

1) Prompt for file paths — ask the user for the Prices CSV (file_prices), Japan Yield Excel (file_japan), and the output CSV path (output_file).

2) Load only needed columns — read DATE, Combo, TotalPrice from the prices CSV and DATE, Japan_Yield from the Japan yield Excel to save memory and I/O.

3) Convert DATE to datetime — parse both DATE columns with pd.to_datetime(..., errors='coerce') so invalid dates become NaT.

4) Drop rows with invalid dates — remove any rows missing a valid DATE to avoid merge issues.

5) Sort both DataFrames by date — ordering before the merge can improve performance and reproducibility.

6) Merge on DATE once — perform an inner merge on DATE to join TotalPrice and Japan_Yield for matching dates. This produces a single, compact DataFrame merged.

7) Free memory — delete the original df_prices and df_japan objects to reduce peak memory usage.

8) Group by Combo and compute correlation — for each Combo, compute the Pearson correlation between TotalPrice and Japan_Yield using a groupby(...).apply(...) pattern and store results in a DataFrame results with column Correlation_with_Japan_Yield.

9) Save results to CSV — write the results DataFrame to the specified output_file without the index.

10) Print completion message — show success messages with the output path.

In [ ]:
#Correlation_with_Japan_Bond_Yield( for each Combo)

import pandas as pd

# ---------------- USER INPUT ----------------
file_prices = input("Enter the path of the Prices CSV file: ").strip()
file_japan = input("Enter the path of the Japan Yield Excel file: ").strip()
output_file = input("Enter the path for the output CSV file: ").strip()

# ---------------- LOAD ONLY REQUIRED COLUMNS ----------------
df_prices = pd.read_csv(file_prices, usecols=['DATE', 'Combo', 'TotalPrice'])
df_japan = pd.read_excel(file_japan, usecols=['DATE', 'Japan_Yield'])

# Convert DATE columns to datetime
df_prices['DATE'] = pd.to_datetime(df_prices['DATE'], errors='coerce')
df_japan['DATE'] = pd.to_datetime(df_japan['DATE'], errors='coerce')

# Drop any rows with invalid dates
df_prices.dropna(subset=['DATE'], inplace=True)
df_japan.dropna(subset=['DATE'], inplace=True)

# Sort once to speed up merge
df_prices.sort_values('DATE', inplace=True)
df_japan.sort_values('DATE', inplace=True)

# Merge once (very efficient)
merged = pd.merge(df_prices, df_japan, on='DATE', how='inner')

# Free memory
del df_prices, df_japan

# ---------------- CALCULATE CORRELATIONS PER COMBO ----------------
results = (
    merged
    .groupby('Combo')[['TotalPrice', 'Japan_Yield']]
    .apply(lambda g: g['TotalPrice'].corr(g['Japan_Yield']))
    .reset_index(name='Correlation_with_Japan_Yield')
)

# Save results
results.to_csv(output_file, index=False)

print("\nOptimized correlation calculation completed.")
print(f"Output saved to: {output_file}")


### Step 13: step-by-step description for the Code:Top_25%_Highest_Correlation_Japan_Bond_Yield

1) Prompt for file paths — ask the user for the correlation CSV (file_corr), the multi-sheet data Excel (file_data), and the output Excel path (output_file). Ensure the output directory exists.

2) Normalize combo strings — define normalize_combo() to split each combo by commas, strip whitespace, sort stock names, and rejoin (so different orderings of the same combo match).

3) Load correlation CSV — read file_corr into df_corr and add Combo_Normalized by applying the normalization function.

4) Keep highest Japan-yield correlation per combo — group df_corr by Combo_Normalized and take the maximum Correlation_with_Japan_Yield for each normalized combo; store as df_corr_grouped.

5) Open multi-sheet data workbook — load file_data via pd.ExcelFile() and get the list of sheet names.

6) Process each sheet — for every sheet:

* read the sheet into df_data;

* add Combo_Normalized using the same normalization;

* merge df_data with df_corr_grouped on Combo_Normalized (left join) to attach the highest correlation value;

* rename Correlation_with_Japan_Yield → Highest_Correlation_Japan_Bond_Yield;

* drop rows without a matching correlation (dropna on the new column);

* remove the helper Combo_Normalized column;

* store the processed DataFrame for later writing.

7) Write output Excel — open an openpyxl ExcelWriter and write each processed sheet to output_file, preserving sheet names.

8) Finish — save and close the workbook; print a completion message indicating the output Excel was generated.

In [1]:
#Top_25%_Highest_Correlation_Japan_Bond_Yield

import pandas as pd
import os

# ---------------- USER INPUT ----------------
file_corr = input("Enter the path of the Correlation CSV file: ").strip()
file_data = input("Enter the path of the data Excel file (multiple sheets): ").strip()
output_file = input("Enter the path for the output Excel file: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# -------- Normalize combo sorting -------
def normalize_combo(combo):
    if pd.isna(combo):
        return combo
    stocks = [s.strip() for s in combo.split(",")]
    return ", ".join(sorted(stocks))

# -------- Load correlation CSV --------
print("Loading correlation CSV file...")
df_corr = pd.read_csv(file_corr)
df_corr["Combo_Normalized"] = df_corr["Combo"].apply(normalize_combo)

# -------- Take highest Japan Yield correlation --------
df_corr_grouped = (
    df_corr.groupby("Combo_Normalized", as_index=False)["Correlation_with_Japan_Yield"]
           .max()
)

# -------- Load multi-sheet Excel --------
print("Loading data file with multiple sheets...")
xls_data = pd.ExcelFile(file_data)
sheet_names = xls_data.sheet_names

output_sheets = {}

# -------- Process each sheet --------
for sheet in sheet_names:
    print(f"Processing sheet: {sheet} ...")
    df_data = pd.read_excel(xls_data, sheet_name=sheet)

    # Normalize combo in data
    df_data["Combo_Normalized"] = df_data["Combo"].apply(normalize_combo)

    # Merge highest correlation
    df_merged = df_data.merge(
        df_corr_grouped,
        on="Combo_Normalized",
        how="left"
    )

    # Rename output column
    df_merged.rename(
        columns={"Correlation_with_Japan_Yield": "Highest_Correlation_Japan_Bond_Yield"},
        inplace=True
    )

    # Remove rows where correlation not available
    df_merged = df_merged.dropna(subset=["Highest_Correlation_Japan_Bond_Yield"])

    # Remove helper column
    df_merged.drop(columns=["Combo_Normalized"], inplace=True)

    output_sheets[sheet] = df_merged

# -------- Save Excel with multiple sheets --------
print("Saving output file with multiple sheets...")
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet, df_out in output_sheets.items():
        df_out.to_excel(writer, sheet_name=sheet, index=False)

print("DONE: Output Excel generated with all sheets updated.")


Enter the path of the Correlation CSV file: E:\Backtesting weekly\Correlation_with_Japan_Bond_Yield.csv
Enter the path of the data Excel file (multiple sheets): E:\Backtesting weekly\Top_25%_Lowest_Average_Correlation.xlsx
Enter the path for the output Excel file: E:\Backtesting weekly\Top_25%_Highest_Correlation_Japan_Bond_Yield.xlsx
Loading correlation CSV file...
Loading data file with multiple sheets...
Processing sheet: Common_100_45Days ...
Processing sheet: Common_101_45Days ...
Processing sheet: Common_102_45Days ...
Processing sheet: Common_103_45Days ...
Processing sheet: Common_104_45Days ...
Processing sheet: Common_105_45Days ...
Processing sheet: Common_106_45Days ...
Processing sheet: Common_107_45Days ...
Processing sheet: Common_108_45Days ...
Processing sheet: Common_109_45Days ...
Processing sheet: Common_10_45Days ...
Processing sheet: Common_110_45Days ...
Processing sheet: Common_111_45Days ...
Processing sheet: Common_112_45Days ...
Processing sheet: Common_113_4

Processing sheet: Common_97_45Days ...
Processing sheet: Common_98_45Days ...
Processing sheet: Common_99_45Days ...
Processing sheet: Common_9_45Days ...
Saving output file with multiple sheets...
DONE: Output Excel generated with all sheets updated.


### Step 14: step-by-step description for the Code:Top_50_Combo_Highest_Correlation_Japan_Bond_Yield

1) Prompt for file paths — ask the user for the input multi-sheet Excel (input_excel) and the output Excel (output_excel).

2) Load workbook — open the input Excel with pd.ExcelFile() and read the list of sheet names.

3) Prepare container — create an empty output_sheets dict to hold the processed DataFrame for each sheet.

4) Loop over sheets — iterate through every sheet name and print which sheet is being processed.

5) Read sheet into DataFrame — load the current sheet with pd.read_excel().

6) Check required column — if the sheet does not contain "Highest_Correlation_Japan_Bond_Yield", skip it and print a message.

7) Sort by correlation — sort the sheet in descending order by "Highest_Correlation_Japan_Bond_Yield" so highest correlations appear first.

8) Keep unique combos — remove duplicate Combo rows, keeping only the first (highest-correlation) occurrence for each combo.

9) Select top 50 — take the first 50 rows from the deduplicated DataFrame (df_unique.head(50)).

10) Store sheet result — save the top-50 DataFrame in output_sheets using the same sheet name as the key.

11) Write output workbook — open an openpyxl ExcelWriter and write every processed sheet from output_sheets into the output Excel file.

12) Finish — close/save the writer and print a completion message: top 50 unique combinations exported for every processed sheet.

In [14]:
#Top_10_Combo_Highest_Correlation_Japan_Bond_Yield

import pandas as pd
import os

# ---------------- USER INPUT ----------------
input_excel = input("Enter the input Excel file path (multiple sheets): ").strip()
output_excel = input("Enter the output Excel file path: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_excel), exist_ok=True)

REQUIRED_COL = "Highest_Correlation_Japan_Bond_Yield"
COMBO_COL = "Combo"
TOP_N = 10

print("Loading input Excel file...")
xls = pd.ExcelFile(input_excel)

valid_sheet_count = 0

with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    for sheet in xls.sheet_names:
        print(f"\nProcessing sheet: {sheet}")

        df = pd.read_excel(xls, sheet_name=sheet)

        # ---- Column validation ----
        if REQUIRED_COL not in df.columns or COMBO_COL not in df.columns:
            print(f" Skipped → Missing required columns")
            continue

        # ---- Sort & deduplicate ----
        df_sorted = (
            df.sort_values(REQUIRED_COL, ascending=False)
              .drop_duplicates(subset=COMBO_COL, keep="first")
        )

        # ---- Check top 10 availability ----
        if len(df_sorted) < TOP_N:
            print(f" Skipped → Only {len(df_sorted)} unique combos (need {TOP_N})")
            continue

        # ---- Take top 10 ----
        df_top = df_sorted.head(TOP_N)

        # ---- Write sheet ----
        df_top.to_excel(writer, sheet_name=sheet, index=False)
        valid_sheet_count += 1
        print(" Sheet written")

# ---- Final status ----
if valid_sheet_count == 0:
    print("\n No sheets met the criteria. Output file may be empty.")
else:
    print(f"\n DONE: {valid_sheet_count} valid sheets written successfully.")


Enter the input Excel file path (multiple sheets): C:\swarupa\Top_25%_Highest_Correlation_Japan_Bond_Yield.xlsx
Enter the output Excel file path: C:\swarupa\Top_10_Combo_Highest_Correlation_Japan_Bond_Yield.xlsx
Loading input Excel file...
Processing sheet: Common_100_45Days ...
Processing sheet: Common_101_45Days ...
Processing sheet: Common_102_45Days ...
Processing sheet: Common_103_45Days ...
Processing sheet: Common_104_45Days ...
Processing sheet: Common_105_45Days ...
Processing sheet: Common_106_45Days ...
Processing sheet: Common_107_45Days ...
Processing sheet: Common_108_45Days ...
Processing sheet: Common_109_45Days ...
Processing sheet: Common_10_45Days ...
Processing sheet: Common_110_45Days ...
Processing sheet: Common_111_45Days ...
Processing sheet: Common_112_45Days ...
Processing sheet: Common_113_45Days ...
Processing sheet: Common_114_45Days ...
Processing sheet: Common_115_45Days ...
Processing sheet: Common_116_45Days ...
Processing sheet: Common_117_45Days ...
P

Saving output Excel...
DONE: Top 10 unique combinations exported for every sheet.


### Step 15:Plotting_Top_50_Combo_Highest_Correlation_Japan_Bond_Yield 




In [ ]:
#only for one csv file 

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from collections import defaultdict

# ================== INPUT FILES ==================
full_data_file = r"D:\Trading Stratergies\output_with_EURINR_50DMA_EURINR.csv"
latest_data_file = input("Enter latest CSV file path: ").strip()
excel_file = input("Enter Excel file path: ").strip()
output_pdf = r"C:\Users\Swarupa\Downloads\Plots_eurinr.pdf"

# ================== LOAD EXCEL ==================
print("Loading Excel file...")
df_excel = pd.read_excel(
    excel_file,
    usecols=[
        "DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR",
        "Average_Realized_Correlation",
        "Highest_Correlation_Japan_Bond_Yield"
    ]
)
df_excel["DATE"] = pd.to_datetime(df_excel["DATE"])
valid_combos = set(df_excel["Combo"])
print(f"Total combos in Excel: {len(valid_combos)}")

# ================== STREAM FULL CSV ==================
print("Streaming FULL historical CSV...")
combo_full = defaultdict(list)

for chunk in tqdm(
    pd.read_csv(
        full_data_file,
        usecols=["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR"],
        parse_dates=["DATE"],
        chunksize=50000
    ),
    desc="Reading FULL CSV",
    unit="chunk"
):
    chunk = chunk[chunk["Combo"].isin(valid_combos)]
    for combo, group in chunk.groupby("Combo"):
        combo_full[combo].append(group)

# ================== LOAD LATEST CSV ==================
print("Loading latest CSV...")
df_latest = pd.read_csv(
    latest_data_file,
    parse_dates=["DATE"],
    usecols=["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR"]
)

df_latest = df_latest[df_latest["Combo"].isin(valid_combos)]

# group latest file by Combo
combo_latest = {c: d for c, d in df_latest.groupby("Combo")}

# ================== PLOTTING ==================
print("\nGenerating plots...\n")
plot_count = 0

with PdfPages(output_pdf) as pdf:
    for combo in tqdm(valid_combos, desc="Plotting", unit="combo"):

        if combo not in combo_full:
            print(f"⚠ Warning: Combo {combo} not found in FULL CSV.")
            continue

        full_df = pd.concat(combo_full[combo]).sort_values("DATE")
        latest_df = combo_latest.get(combo, pd.DataFrame())
        excel_df = df_excel[df_excel["Combo"] == combo]

        # ------------------ PLOT ------------------
        plt.figure(figsize=(12, 7))

        # FULL FILE → Light color
        plt.plot(full_df["DATE"], full_df["EURINR_Ratio"],
                 label="Full EURINR_Ratio", color="lightblue", linewidth=1)
        plt.plot(full_df["DATE"], full_df["50DMA_EURINR"],
                 label="Full 50DMA_EURINR", color="lightgray",
                 linestyle="--", linewidth=1)

        # LATEST FILE → Dark bold color
        if not latest_df.empty:
            plt.plot(latest_df["DATE"], latest_df["EURINR_Ratio"],
                     label="Latest EURINR_Ratio", color="blue", linewidth=1)
            plt.plot(latest_df["DATE"], latest_df["50DMA_EURINR"],
                     label="Latest 50DMA_EURINR", color="black", linestyle="--", linewidth=1)

        # Excel points
        plt.scatter(excel_df["DATE"], excel_df["EURINR_Ratio"],
                    s=80, marker="o", color="red", label="Excel EURINR_Ratio")
        plt.scatter(excel_df["DATE"], excel_df["50DMA_EURINR"],
                    s=80, marker="x", color="green", label="Excel 50DMA_EURINR")
        plt.scatter(excel_df["DATE"], excel_df["Average_Realized_Correlation"],
                    s=120, marker="D", color="purple",
                    label="Avg Realized Corr")
        plt.scatter(excel_df["DATE"], excel_df["Highest_Correlation_Japan_Bond_Yield"],
                    s=120, marker="s", color="orange",
                    label="Highest Japan Bond Yield")

        plt.title(f"EURINR Plot – {combo}")
        plt.xlabel("Date")
        plt.ylabel("Value")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()

        pdf.savefig()
        plt.close()
        plot_count += 1

print("\n===========================")
print(f"Total plots created: {plot_count}")
print(f"Output saved to: {output_pdf}")
print("===========================\n")


In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
from collections import defaultdict

# ================= USER INPUT =================
FULL_CSV = r"C:\swarupa\Trading Stratergies\output_with_EURINR_50DMA_EURINR.csv"
EXCEL_FILE = r"C:\swarupa\Top_10_Combo_Highest_Correlation_Japan_Bond_Yield.xlsx"
PARQUET_FOLDER = input("Enter Parquet folder path: ").strip()
OUTPUT_FOLDER = input("Enter Output PDF folder path: ").strip()

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ================= LOAD EXCEL (ALL SHEETS) =================
print("Loading Excel...")
xls = pd.ExcelFile(EXCEL_FILE)

excel_by_sheet = {}
excel_combos = set()

for sheet in xls.sheet_names:
    df = pd.read_excel(
        EXCEL_FILE,
        sheet_name=sheet,
        usecols=[
            "DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR",
            "Average_Realized_Correlation",
            "Highest_Correlation_Japan_Bond_Yield"
        ]
    )
    df["DATE"] = pd.to_datetime(df["DATE"])
    excel_by_sheet[sheet] = df.groupby("Combo", sort=False)
    excel_combos.update(df["Combo"].unique())

print(f"Sheets: {len(excel_by_sheet)} | Combos: {len(excel_combos)}")

# ================= STREAM FULL CSV (ONE PASS, LOW RAM) =================
print("Streaming FULL CSV...")
combo_full = defaultdict(list)

for chunk in tqdm(
    pd.read_csv(
        FULL_CSV,
        usecols=["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR"],
        parse_dates=["DATE"],
        chunksize=150_000
    ),
    desc="Reading CSV"
):
    chunk = chunk[chunk["Combo"].isin(excel_combos)]
    for combo, grp in chunk.groupby("Combo", sort=False):
        combo_full[combo].append(grp)

# Final compact DataFrames
combo_full = {
    c: pd.concat(v, ignore_index=True).sort_values("DATE")
    for c, v in combo_full.items()
}

# ================= PARQUET FILES =================
parquet_files = sorted(glob.glob(os.path.join(PARQUET_FOLDER, "*.parquet")))
print(f"Parquet files: {len(parquet_files)}")

# ================= PROCESS EACH PARQUET =================
for pq_file in tqdm(parquet_files, desc="Processing Parquet", unit="file"):

    pdf_path = os.path.join(
        OUTPUT_FOLDER,
        os.path.splitext(os.path.basename(pq_file))[0] + ".pdf"
    )

    print(f"\nCreating PDF: {pdf_path}")

    # ---- Read parquet (columns only) ----
    df_pq = pd.read_parquet(
        pq_file,
        columns=["DATE", "DATE_INT", "Combo", "Combo_ID", "EURINR_Ratio", "50DMA_EURINR"],
        engine="pyarrow"
    )

    # Normalize columns
    if "DATE" not in df_pq.columns:
        df_pq["DATE"] = pd.to_datetime(
            df_pq["DATE_INT"].astype(str), format="%Y%m%d", errors="coerce"
        )

    if "Combo" not in df_pq.columns:
        df_pq.rename(columns={"Combo_ID": "Combo"}, inplace=True)

    df_pq = df_pq[["DATE", "Combo", "EURINR_Ratio", "50DMA_EURINR"]]
    pq_grouped = df_pq.groupby("Combo", sort=False)

    # ================= PLOTTING =================
    with PdfPages(pdf_path) as pdf:

        for sheet, grp_excel in excel_by_sheet.items():

            for combo, excel_df in grp_excel:

                full_df = combo_full.get(combo)
                if full_df is None:
                    continue

                pq_df = pq_grouped.get_group(combo) if combo in pq_grouped.groups else None

                fig, ax = plt.subplots(figsize=(11, 6))

                # FULL CSV
                ax.plot(full_df["DATE"], full_df["EURINR_Ratio"],
                        color="lightblue", linewidth=0.8, label="Full EURINR")
                ax.plot(full_df["DATE"], full_df["50DMA_EURINR"],
                        color="lightgray", linestyle="--", linewidth=0.8,
                        label="Full 50DMA")

                # PARQUET
                if pq_df is not None:
                    ax.plot(pq_df["DATE"], pq_df["EURINR_Ratio"],
                            color="blue", linewidth=1.2, label="Parquet EURINR")
                    ax.plot(pq_df["DATE"], pq_df["50DMA_EURINR"],
                            color="black", linestyle="--", linewidth=1.2,
                            label="Parquet 50DMA")

                # EXCEL POINTS
                ax.scatter(excel_df["DATE"], excel_df["EURINR_Ratio"],
                           s=50, color="red", label="Excel EURINR")
                ax.scatter(excel_df["DATE"], excel_df["50DMA_EURINR"],
                           s=50, marker="x", color="green", label="Excel 50DMA")
                ax.scatter(excel_df["DATE"], excel_df["Average_Realized_Correlation"],
                           s=70, marker="D", color="purple", label="Avg Corr")
                ax.scatter(excel_df["DATE"],
                           excel_df["Highest_Correlation_Japan_Bond_Yield"],
                           s=70, marker="s", color="orange", label="JP Bond")

                ax.set_title(f"{sheet} | {combo}")
                ax.grid(True)
                ax.legend()
                fig.tight_layout()

                pdf.savefig(fig)
                plt.close(fig)

    print(f"Saved → {pdf_path}")


Enter Parquet folder path: D:\EURINR_50DMA
Enter Output PDF folder path: D:\Eurinr_Plots
Loading Excel...
Sheets: 202 | Combos: 274
Streaming FULL CSV...


Reading CSV: 454it [01:47,  4.23it/s]

In [2]:
import pandas as pd
import os
import glob

# ---------------- USER INPUT ----------------
file_corr = input("Enter the path of the Correlation Excel file: ").strip()
csv_folder = input("Enter the path of the CSV folder: ").strip()
output_file = input("Enter the path for the output Excel file: ").strip()

LOWEST_PERCENT = float(
    input("Enter percentage of LOWEST Average Correlation to keep (e.g. 25): ")
) / 100

os.makedirs(os.path.dirname(output_file), exist_ok=True)

# ---------------- FUNCTIONS ----------------
def normalize_combo_series(series: pd.Series) -> pd.Series:
    """Normalize combo strings by sorting stock names."""
    return (
        series
        .dropna()
        .str.split(",")
        .apply(lambda x: ", ".join(sorted(s.strip() for s in x)))
        .reindex(series.index)
    )

# ---------------- LOAD CORRELATION FILE ----------------
print("Loading correlation file...")
df_corr = pd.read_excel(
    file_corr,
    usecols=["Combo", "Average_Realized_Correlation"]
)

df_corr["Combo_Normalized"] = normalize_combo_series(df_corr["Combo"])
df_corr = df_corr[["Combo_Normalized", "Average_Realized_Correlation"]]

# ---------------- PROCESS CSV FILES ----------------
csv_files = sorted(glob.glob(os.path.join(csv_folder, "*.csv")))

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for csv_file in csv_files:
        sheet_name = os.path.splitext(os.path.basename(csv_file))[0][:31]
        print(f"Processing: {sheet_name}")

        df_data = pd.read_csv(csv_file)

        # Normalize combo
        df_data["Combo_Normalized"] = normalize_combo_series(df_data["Combo"])

        # Merge correlation
        df_merged = df_data.merge(
            df_corr,
            on="Combo_Normalized",
            how="left"
        )

        # Drop rows without correlation
        df_merged.dropna(
            subset=["Average_Realized_Correlation"],
            inplace=True
        )

        # ---------------- LOWEST X% CORRELATION ----------------
        cutoff = df_merged["Average_Realized_Correlation"].quantile(LOWEST_PERCENT)
        df_merged = df_merged[
            df_merged["Average_Realized_Correlation"] <= cutoff
        ]

        # Cleanup
        df_merged.drop(columns=["Combo_Normalized"], inplace=True)

        # Save as sheet
        df_merged.to_excel(writer, sheet_name=sheet_name, index=False)

print(
    f"DONE: Lowest {LOWEST_PERCENT * 100:.0f}% Average Correlation saved per CSV file."
)


Enter the path of the Correlation Excel file: C:\swarupa\Trading Stratergies\4Stock_Combinations_Correlation.xlsx
Enter the path of the CSV folder: C:\swarupa\Ratio_GT_50DMA
Enter the path for the output Excel file: D:\Testing\Top_25%_Lowest_Average_Correlation.xlsx
Enter percentage of LOWEST Average Correlation to keep (e.g. 25): 25
Loading correlation file...
Processing: Common_100_45Days
Processing: Common_101_45Days
Processing: Common_102_45Days
Processing: Common_103_45Days
Processing: Common_104_45Days
Processing: Common_105_45Days
Processing: Common_106_45Days
Processing: Common_107_45Days
Processing: Common_108_45Days
Processing: Common_109_45Days
Processing: Common_10_45Days
Processing: Common_110_45Days
Processing: Common_111_45Days
Processing: Common_112_45Days
Processing: Common_113_45Days
Processing: Common_114_45Days
Processing: Common_115_45Days
Processing: Common_116_45Days
Processing: Common_117_45Days
Processing: Common_118_45Days
Processing: Common_119_45Days
Process

In [3]:
#Top_25%_Highest_Correlation_Japan_Bond_Yield

import pandas as pd
import os

# ---------------- USER INPUT ----------------
file_corr = input("Enter the path of the Correlation CSV file: ").strip()
file_data = input("Enter the path of the data Excel file (multiple sheets): ").strip()
output_file = input("Enter the path for the output Excel file: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# -------- Normalize combo sorting -------
def normalize_combo(combo):
    if pd.isna(combo):
        return combo
    stocks = [s.strip() for s in combo.split(",")]
    return ", ".join(sorted(stocks))

# -------- Load correlation CSV --------
print("Loading correlation CSV file...")
df_corr = pd.read_csv(file_corr)
df_corr["Combo_Normalized"] = df_corr["Combo"].apply(normalize_combo)

# -------- Take highest Japan Yield correlation --------
df_corr_grouped = (
    df_corr.groupby("Combo_Normalized", as_index=False)["Correlation_with_Japan_Yield"]
           .max()
)

# -------- Load multi-sheet Excel --------
print("Loading data file with multiple sheets...")
xls_data = pd.ExcelFile(file_data)
sheet_names = xls_data.sheet_names

output_sheets = {}

# -------- Process each sheet --------
for sheet in sheet_names:
    print(f"Processing sheet: {sheet} ...")
    df_data = pd.read_excel(xls_data, sheet_name=sheet)

    # Normalize combo in data
    df_data["Combo_Normalized"] = df_data["Combo"].apply(normalize_combo)

    # Merge highest correlation
    df_merged = df_data.merge(
        df_corr_grouped,
        on="Combo_Normalized",
        how="left"
    )

    # Rename output column
    df_merged.rename(
        columns={"Correlation_with_Japan_Yield": "Highest_Correlation_Japan_Bond_Yield"},
        inplace=True
    )

    # Remove rows where correlation not available
    df_merged = df_merged.dropna(subset=["Highest_Correlation_Japan_Bond_Yield"])

    # Remove helper column
    df_merged.drop(columns=["Combo_Normalized"], inplace=True)

    output_sheets[sheet] = df_merged

# -------- Save Excel with multiple sheets --------
print("Saving output file with multiple sheets...")
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet, df_out in output_sheets.items():
        df_out.to_excel(writer, sheet_name=sheet, index=False)

print("DONE: Output Excel generated with all sheets updated.")


Enter the path of the Correlation CSV file: C:\swarupa\Correlation_with_Japan_Bond_Yield.csv
Enter the path of the data Excel file (multiple sheets): D:\Testing\Top_25%_Lowest_Average_Correlation.xlsx
Enter the path for the output Excel file: D:\Testing\Top_25%_Highest_Correlation_Japan_Bond_Yield.xlsx
Loading correlation CSV file...
Loading data file with multiple sheets...
Processing sheet: Common_100_45Days ...
Processing sheet: Common_101_45Days ...
Processing sheet: Common_102_45Days ...
Processing sheet: Common_103_45Days ...
Processing sheet: Common_104_45Days ...
Processing sheet: Common_105_45Days ...
Processing sheet: Common_106_45Days ...
Processing sheet: Common_107_45Days ...
Processing sheet: Common_108_45Days ...
Processing sheet: Common_109_45Days ...
Processing sheet: Common_10_45Days ...
Processing sheet: Common_110_45Days ...
Processing sheet: Common_111_45Days ...
Processing sheet: Common_112_45Days ...
Processing sheet: Common_113_45Days ...
Processing sheet: Commo

Processing sheet: Common_97_45Days ...
Processing sheet: Common_98_45Days ...
Processing sheet: Common_99_45Days ...
Processing sheet: Common_9_45Days ...
Saving output file with multiple sheets...
DONE: Output Excel generated with all sheets updated.


In [24]:
#Top_10_Combo_Highest_Correlation_Japan_Bond_Yield

import pandas as pd
import os

# ---------------- USER INPUT ----------------
input_excel = input("Enter the input Excel file path (multiple sheets): ").strip()
output_excel = input("Enter the output Excel file path: ").strip()

# Ensure output directory exists
os.makedirs(os.path.dirname(output_excel), exist_ok=True)

REQUIRED_COL = "Highest_Correlation_Japan_Bond_Yield"
COMBO_COL = "Combo"
TOP_N = 20

print("Loading input Excel file...")
xls = pd.ExcelFile(input_excel)

valid_sheet_count = 0

with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    for sheet in xls.sheet_names:
        print(f"\nProcessing sheet: {sheet}")

        df = pd.read_excel(xls, sheet_name=sheet)

        # ---- Column validation ----
        if REQUIRED_COL not in df.columns or COMBO_COL not in df.columns:
            print(f" Skipped → Missing required columns")
            continue

        # ---- Sort & deduplicate ----
        df_sorted = (
            df.sort_values(REQUIRED_COL, ascending=False)
              .drop_duplicates(subset=COMBO_COL, keep="first")
        )

        # ---- Check top 10 availability ----
        if len(df_sorted) < TOP_N:
            print(f" Skipped → Only {len(df_sorted)} unique combos (need {TOP_N})")
            continue

        # ---- Take top 10 ----
        df_top = df_sorted.head(TOP_N)

        # ---- Write sheet ----
        df_top.to_excel(writer, sheet_name=sheet, index=False)
        valid_sheet_count += 1
        print(" Sheet written")

# ---- Final status ----
if valid_sheet_count == 0:
    print("\n No sheets met the criteria. Output file may be empty.")
else:
    print(f"\n DONE: {valid_sheet_count} valid sheets written successfully.")


Enter the input Excel file path (multiple sheets): C:\swarupa\Top_25%_Highest_Correlation_Japan_Bond_Yield.xlsx
Enter the output Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\Top_20_HCJBY.xlsx
Loading input Excel file...

Processing sheet: Common_100_45Days
 Sheet written

Processing sheet: Common_101_45Days
 Sheet written

Processing sheet: Common_102_45Days
 Sheet written

Processing sheet: Common_103_45Days
 Sheet written

Processing sheet: Common_104_45Days
 Sheet written

Processing sheet: Common_105_45Days
 Sheet written

Processing sheet: Common_106_45Days
 Sheet written

Processing sheet: Common_107_45Days
 Skipped → Only 0 unique combos (need 20)

Processing sheet: Common_108_45Days
 Sheet written

Processing sheet: Common_109_45Days
 Sheet written

Processing sheet: Common_10_45Days
 Sheet written

Processing sheet: Common_110_45Days
 Sheet written

Processing sheet: Common_111_45Days
 Sheet written

Processing sheet: Common_112_45Days
 Sheet written

Proces

 Sheet written

Processing sheet: Common_57_45Days
 Sheet written

Processing sheet: Common_58_45Days
 Sheet written

Processing sheet: Common_59_45Days
 Sheet written

Processing sheet: Common_5_45Days
 Sheet written

Processing sheet: Common_60_45Days
 Sheet written

Processing sheet: Common_61_45Days
 Sheet written

Processing sheet: Common_62_45Days
 Sheet written

Processing sheet: Common_63_45Days
 Sheet written

Processing sheet: Common_64_45Days
 Sheet written

Processing sheet: Common_65_45Days
 Sheet written

Processing sheet: Common_66_45Days
 Sheet written

Processing sheet: Common_67_45Days
 Sheet written

Processing sheet: Common_68_45Days
 Sheet written

Processing sheet: Common_69_45Days
 Sheet written

Processing sheet: Common_6_45Days
 Sheet written

Processing sheet: Common_70_45Days
 Sheet written

Processing sheet: Common_71_45Days
 Sheet written

Processing sheet: Common_72_45Days
 Sheet written

Processing sheet: Common_73_45Days
 Skipped → Only 0 unique combos (

In [1]:
#only_select_combos

import pandas as pd
import os
import re

# ================= USER INPUT =================
INPUT_FILE  = r"C:\swarupa\performance_gain_loss\Top_10_Combos\Top_10_HCJBY.xlsx"
OUTPUT_FILE = r"C:\swarupa\performance_gain_loss\Top_10_Combos\only_select_10combos.xlsx"

# ================= LOAD ALL SHEETS =================
xls = pd.ExcelFile(INPUT_FILE)

os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

def extract_number(sheet_name):
    """
    Extracts the numeric part from Common_<number>_45Days
    """
    match = re.search(r"Common_(\d+)_", sheet_name)
    return int(match.group(1)) if match else float("inf")

# Sort sheet names numerically
sorted_sheets = sorted(xls.sheet_names, key=extract_number)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:
    for sheet_name in sorted_sheets:
        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Keep ONLY Combo column
        out_df = df[["Combo"]] if "Combo" in df.columns else pd.DataFrame({"Combo": []})

        # Write with SAME sheet name
        out_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Output file created with sheets ordered serially (numeric), names unchanged.")


Output file created with sheets ordered serially (numeric), names unchanged.


In [26]:
#Combo_With_other_columns

import pandas as pd

# ================= FILE PATHS =================
FIRST_FILE  = r"C:\swarupa\performance_gain_loss\combo_input_data.xlsx"       
SECOND_FILE = r"C:\swarupa\performance_gain_loss\Top_20_Combos\only_select_20combos.xlsx"
OUTPUT_FILE = r"C:\swarupa\performance_gain_loss\Top_20_Combos\20_Combo_With_other_columns.xlsx"

# ================= LOAD FILES =================
first_xls  = pd.ExcelFile(FIRST_FILE)
second_xls = pd.ExcelFile(SECOND_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:

    for i, sheet_name in enumerate(second_xls.sheet_names):

        # Read files
        first_df  = pd.read_excel(first_xls, sheet_name=first_xls.sheet_names[i])
        second_df = pd.read_excel(second_xls, sheet_name=sheet_name)

        # Identify column names safely
        col2_name = first_df.columns[1]
        col3_name = first_df.columns[2]

        # Keep only required columns from first file
        lookup_df = first_df[["Combo", col2_name, col3_name]]

        # ================= COMBO-BASED MERGE =================
        final_df = second_df.merge(
            lookup_df,
            on="Combo",
            how="left"
        )

        # Write output
        final_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(" Error fixed: columns added using Combo matching.")


 Error fixed: columns added using Combo matching.


In [27]:
#Combo_Top_10_total_price

import os
import pandas as pd
import re
import numpy as np

# ================= USER INPUT =================
COMBO_FILE  = input("Enter Combo Excel file path: ").strip()
PRICE_FILE  = input("Enter Price Excel file path: ").strip()
OUTPUT_FILE = input("Enter Output Excel file path: ").strip()
INVESTMENT  = float(input("Enter investment per company (e.g. 50000): "))

# ================= CLEAN OLD OUTPUT =================
if os.path.exists(OUTPUT_FILE):
    os.remove(OUTPUT_FILE)

# ================= LOAD PRICE DATA =================
price_df = pd.read_excel(PRICE_FILE)
price_df["DATE"] = pd.to_datetime(price_df["DATE"], format="%d-%b-%y")
price_df.set_index("DATE", inplace=True)
price_index = price_df.index

# ================= HELPER FUNCTIONS =================
def extract_date(col_name):
    match = re.search(r"\d{4}-\d{2}-\d{2}", col_name)
    if not match:
        raise ValueError(f"Date not found in column name: {col_name}")
    return pd.to_datetime(match.group())

def nearest_date(index, target):
    return index[index.get_indexer([target], method="nearest")[0]]

# ================= PROCESS COMBO FILE =================
combo_xls = pd.ExcelFile(COMBO_FILE)

with pd.ExcelWriter(OUTPUT_FILE, engine="xlsxwriter") as writer:

    for sheet_name in combo_xls.sheet_names:

        combo_df = pd.read_excel(combo_xls, sheet_name=sheet_name)

        # ---- Extract dates from column names ----
        start_date = extract_date(combo_df.columns[1])
        end_date   = extract_date(combo_df.columns[2])

        if start_date not in price_index:
            start_date = nearest_date(price_index, start_date)
        if end_date not in price_index:
            end_date = nearest_date(price_index, end_date)

        start_prices = price_df.loc[start_date]
        end_prices   = price_df.loc[end_date]

        output_rows = []

        for combo in combo_df["Combo"]:
            companies = combo.split(",")

            # ---- Quantity calculation ----
            quantities = INVESTMENT / start_prices[companies].values

            start_total = int(round(INVESTMENT * len(companies)))
            end_total   = int(round(np.sum(quantities * end_prices[companies].values)))

            output_rows.append({
                "Combo": combo,
                "start_price": start_total,
                "final_price": end_total
            })

        out_df = pd.DataFrame(output_rows)

        #  OUTPUT SHEET NAME = INPUT SHEET NAME
        out_df.to_excel(writer, sheet_name=sheet_name[:31], index=False)

print("\n Process completed successfully")
print(f" Output file created at: {OUTPUT_FILE}")


Enter Combo Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\20_Combo_With_other_columns.xlsx
Enter Price Excel file path: C:\swarupa\Trading Stratergies\50 comp data (input file).xlsx
Enter Output Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\Combo_Top_20_total_price.xlsx
Enter investment per company (e.g. 50000): 90000

 Process completed successfully
 Output file created at: C:\swarupa\performance_gain_loss\Top_20_Combos\Combo_Top_20_total_price.xlsx


In [28]:
# Combo_Top_10_Gain_Loss

import pandas as pd
import numpy as np

# ===================== USER INPUT =====================
input_file = input("Enter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ===================== LOAD INPUT =====================
xls = pd.ExcelFile(input_file)

summary_rows = []
detailed_sheets = {}

# ===================== PROCESS EACH INPUT SHEET =====================
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)

    # -------- FIND PRICE COLUMNS DYNAMICALLY --------
    start_col = next((c for c in df.columns if c.startswith("start_price")), None)
    final_col = next((c for c in df.columns if c.startswith("final_price")), None)

    if start_col is None or final_col is None:
        raise KeyError(
            f"Sheet '{sheet_name}' missing start_price or final_price column"
        )

    # ----- ADD CALCULATION COLUMNS -----
    df["Gain_Loss_Amt"] = df[final_col] - df[start_col]
    df["Gain_Amt"] = np.where(df["Gain_Loss_Amt"] > 0, df["Gain_Loss_Amt"], 0)
    df["Loss_Amt"] = np.where(df["Gain_Loss_Amt"] < 0, abs(df["Gain_Loss_Amt"]), 0)
    df["No_of_Gains"] = np.where(df["Gain_Loss_Amt"] > 0, 1, 0)
    df["No_of_Loss"] = np.where(df["Gain_Loss_Amt"] < 0, 1, 0)

    # ----- SELECT COLUMNS FOR DETAILED SHEET -----
    detailed_df = df[
        ["Combo", start_col, final_col, "Gain_Loss_Amt", "Gain_Amt", "Loss_Amt"]
    ]

    detailed_sheets[sheet_name] = detailed_df

    # ----- CREATE SHEET-WISE SUMMARY -----
    summary_rows.append({
        "Sheet_Name": sheet_name,
        "No_of_Gains": int(df["No_of_Gains"].sum()),
        "No_of_Loss": int(df["No_of_Loss"].sum()),
        "Total_Gains_Amt": round(df["Gain_Amt"].sum(), 2),
        "Total_Loss_Amt": round(df["Loss_Amt"].sum(), 2),
        "Net_Gain_Loss_Amt": round(df["Gain_Amt"].sum() - df["Loss_Amt"].sum(), 2)
    })

# ===================== WRITE OUTPUT =====================
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:

    # Summary first
    summary_df = pd.DataFrame(summary_rows)
    summary_df.to_excel(writer, sheet_name="Summary_All_Sheets", index=False)

    # Detailed sheets
    for sheet_name, df in detailed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(" Output created successfully")


Enter input Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\Combo_Top_20_total_price.xlsx
Enter output Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\Combo_Top_10_Gain_Loss.xlsx
 Output created successfully


In [29]:
#Summary_With_Percentage_Columns

import pandas as pd

# ================= USER INPUT =================
input_file = input("Enter input Excel file path: ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ================= LOAD INPUT  =================
df = pd.read_excel(input_file)

# ================= CONSTANTS =================
CAPITAL = 360000 * 20
TOTAL_COMBOS = 20
TOTAL_WEEKS = len(df)   

# ================= ADD OUTPUT COLUMNS =================
df["%Returns"] = ((df["Net_Gain_Loss_Amt"] / CAPITAL) * 100).round(2)
df["%No_of_Gains"] = ((df["No_of_Gains"] / TOTAL_COMBOS) * 100).round(2)

df["%No_of_Loss"] = ((df["No_of_Loss"] / TOTAL_COMBOS) * 100).round(2)

# ================= OVERALL SUMMARY =================
overall_df = pd.DataFrame({
    "Metric": [
        "%Total_Returns",
        "%Total_No_of_Gains",
        "%Total_No_of_Loss"
    ],
    "Value": [
        round((df["Net_Gain_Loss_Amt"].sum() / (CAPITAL * TOTAL_WEEKS)) * 100,2),
        round((df["No_of_Gains"].sum() / (TOTAL_COMBOS * TOTAL_WEEKS)) * 100,2),
        round((df["No_of_Loss"].sum() / (TOTAL_COMBOS * TOTAL_WEEKS)) * 100,2)
    ]
})

# ================= WRITE OUTPUT =================
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="Weekly_Data", index=False)
    overall_df.to_excel(writer, sheet_name="Overall_Summary", index=False)

print("\n Calculation completed successfully.")
print(" Output file created at:", output_file)


Enter input Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\Only_summary_20xlsx.xlsx
Enter output Excel file path: C:\swarupa\performance_gain_loss\Top_20_Combos\summary_20week.xlsx

 Calculation completed successfully.
 Output file created at: C:\swarupa\performance_gain_loss\Top_20_Combos\summary_20week.xlsx


In [3]:
#NIFTY_Performance_output_in_single_fie

import pandas as pd
import numpy as np

# ================= USER INPUT =================
nifty_file  = input("Enter NIFTY Excel file path: ").strip()
combo_file  = input("Enter Combo Excel file path (multiple sheets): ").strip()
output_file = input("Enter Output Excel file path: ").strip()

# ================= LOAD NIFTY DATA =================
nifty_df = pd.read_excel(nifty_file)
nifty_df['DATE'] = pd.to_datetime(nifty_df['DATE'])
nifty_df.set_index('DATE', inplace=True)
nifty_df.sort_index(inplace=True)

# ---------- Helper ----------
def get_nearest_nifty_value(target_date):
    if target_date in nifty_df.index:
        return nifty_df.loc[target_date, 'NIFTY']

    nearest_idx = nifty_df.index.get_indexer(
        [target_date], method='nearest'
    )[0]

    return nifty_df.iloc[nearest_idx]['NIFTY']

# ================= PROCESS MULTIPLE SHEETS =================
xls = pd.ExcelFile(combo_file)

summary_rows = []

for sheet_name in xls.sheet_names:

    df = pd.read_excel(xls, sheet_name=sheet_name)

    # Identify start & end date columns
    start_col = [c for c in df.columns if c.startswith("start_date")][0]
    end_col   = [c for c in df.columns if c.startswith("end_date")][0]

    # Extract dates from column headers
    start_date = pd.to_datetime(start_col.replace("start_date_", ""))
    end_date   = pd.to_datetime(end_col.replace("end_date_", ""))

    # Fetch NIFTY values
    start_nifty = get_nearest_nifty_value(start_date)
    end_nifty   = get_nearest_nifty_value(end_date)

    # Calculate %Returns 
    returns = (
        (((end_nifty - start_nifty) / start_nifty) * 100 ,2)
        if start_nifty != 0 else np.nan
    )

    summary_rows.append({
        'Sheet_Name': sheet_name,
        'NIFTY_Start': start_nifty,
        'NIFTY_Final': end_nifty,
        '%Returns': returns
    })

# ================= SAVE SUMMARY FILE =================
summary_df = pd.DataFrame(summary_rows)
summary_df.to_excel(output_file, index=False)

print("\n summary file generated successfully.")


Enter NIFTY Excel file path: E:\Backtesting_jan_2020_to_19_aug_2025\Nifty_Index_values(input file).xlsx
Enter Combo Excel file path (multiple sheets): C:\swarupa\performance_gain_loss\Top_10_Combos\10_Combo_With_other_columns.xlsx
Enter Output Excel file path: C:\swarupa\performance_gain_loss\Top_10_Combos\NIFTY_Performance.xlsx

 summary file generated successfully.


In [ ]:
#returns_comparison_on_our_data_nifty_data

import pandas as pd

# ================= FILE PATHS =================
nifty_file = r"C:\swarupa\performance_gain_loss\Top_10_Combos\NIFTY_Performance_202_Weeks.xlsx"
ourdata_file = r"C:\Users\Swarupa\Downloads\Top_10_summary_202week.xlsx"

# ================= COLUMN NAME =================
return_col = "%Returns"   # change here if needed

# ================= LOAD NIFTY DATA =================
nifty_df = pd.read_excel(nifty_file)

# Ensure numeric
nifty_df[return_col] = pd.to_numeric(nifty_df[return_col], errors="coerce")

nifty_gain_count = (nifty_df[return_col] > 0).sum()
nifty_loss_count = (nifty_df[return_col] <= 0).sum()
nifty_total = nifty_df[return_col].count()

# ================= LOAD OUR DATA (ALL SHEETS) =================
xls = pd.ExcelFile(ourdata_file)

our_dfs = []
for sheet in xls.sheet_names:
    df = pd.read_excel(ourdata_file, sheet_name=sheet)
    if return_col in df.columns:
        df[return_col] = pd.to_numeric(df[return_col], errors="coerce")
        df["Sheet_Name"] = sheet
        our_dfs.append(df)

our_df = pd.concat(our_dfs, ignore_index=True)

our_gain_count = (our_df[return_col] > 0).sum()
our_loss_count = (our_df[return_col] <= 0).sum()
our_total = our_df[return_col].count()

# ================= SUMMARY TABLE =================
summary = pd.DataFrame({
    "Metric": ["Total Observations", "Positive Returns Count", "Negative / Zero Returns Count"],
    "NIFTY": [nifty_total, nifty_gain_count, nifty_loss_count],
    "OUR_DATA": [our_total, our_gain_count, our_loss_count]
})

print("\n===== PERFORMANCE COMPARISON =====\n")
print(summary)

# ================= OPTIONAL: SAVE RESULT =================
output_file = r"C:\Users\Swarupa\Downloads\Return_Comparison_Summary_202.xlsx"
summary.to_excel(output_file, index=False)

print(f"\nSummary saved to: {output_file}")


In [1]:
#NIFTY_Performance_output_in_multiple_sheets

import pandas as pd
import numpy as np

# ================= USER INPUT =================
nifty_file  = input("Enter NIFTY Excel file path: ").strip()
combo_file  = input("Enter Combo Excel file path (multiple sheets): ").strip()
output_file = input("Enter Output Excel file path: ").strip()

# ================= LOAD NIFTY DATA =================
nifty_df = pd.read_excel(nifty_file, usecols=['DATE', 'NIFTY'])
nifty_df['DATE'] = pd.to_datetime(nifty_df['DATE'])
nifty_df = nifty_df.sort_values('DATE').set_index('DATE')

nifty_index  = nifty_df.index
nifty_values = nifty_df['NIFTY'].values

# ---------- FAST NEAREST VALUE FUNCTION ----------
def get_nearest_nifty_value(target_date):
    pos = nifty_index.searchsorted(target_date)

    if pos == 0:
        return nifty_values[0]
    if pos == len(nifty_index):
        return nifty_values[-1]

    before = nifty_index[pos - 1]
    after  = nifty_index[pos]

    return nifty_values[pos - 1] if abs(target_date - before) <= abs(after - target_date) else nifty_values[pos]

# ================= PROCESS MULTIPLE SHEETS =================
xls = pd.ExcelFile(combo_file)

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:

    sheet_counter = 1   

    for sheet_name in xls.sheet_names:

        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Identify start & end date columns
        start_cols = [c for c in df.columns if c.startswith("start_date_")]
        end_cols   = [c for c in df.columns if c.startswith("end_date_")]

        if not start_cols or not end_cols:
            continue

        # Extract dates from column names
        start_date = pd.to_datetime(start_cols[0].replace("start_date_", ""))
        end_date   = pd.to_datetime(end_cols[0].replace("end_date_", ""))

        # Fetch NIFTY values
        start_nifty = get_nearest_nifty_value(start_date)
        end_nifty   = get_nearest_nifty_value(end_date)

        # Calculate %Returns
        returns = round(((end_nifty - start_nifty) / start_nifty) * 100, 2)

        # Create output dataframe
        output_df = pd.DataFrame([{
            'NIFTY_Start': start_nifty,
            'NIFTY_Final': end_nifty,
            '%Returns': returns
        }])

        #  New sheet name format
        output_sheet_name = f"Common_{sheet_counter}_45Days"

        output_df.to_excel(writer, sheet_name=output_sheet_name, index=False)

        sheet_counter += 1

print("\n Output generated with sheet names like: Common_1_45Days, Common_2_45Days, ...")


Enter NIFTY Excel file path: C:\swarupa\Nifty_Index_values(input file).xlsx
Enter Combo Excel file path (multiple sheets): C:\swarupa\performance_gain_loss\Combo_date_input_data.xlsx
Enter Output Excel file path: C:\swarupa\performance_gain_loss\Top_10_Combos\NIFTY_performace_multiple_sheets_202.xlsx

 Output generated with sheet names like: Common_1_45Days, Common_2_45Days, ...


In [2]:
#METRIC_PERFORMANCE

import pandas as pd
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment

# ================= USER INPUT =================
input_file_1 = input("Enter FIRST input Excel file (multi-sheet combos): ").strip()
input_file_2 = input("Enter SECOND input Excel file (METRICS): ").strip()
nifty_file   = input("Enter NIFTY Excel file (multi-sheet): ").strip()
output_file  = input("Enter output Excel file path: ").strip()

INVESTMENT_AMOUNT = 90000

# ================= STYLES =================
thin = Side(style="thin")
border = Border(left=thin, right=thin, top=thin, bottom=thin)

header_font  = Font(bold=True, size=13)
bold_font    = Font(bold=True)
center_align = Alignment(horizontal="center", vertical="center")
total_fill   = PatternFill("solid", fgColor="FFFF99")

# ================= HELPERS =================
def normalize(name):
    return name.strip().lower().replace(" ", "")

def apply_borders(ws, start_row, start_col, rows, cols):
    for r in range(start_row, start_row + rows):
        for c in range(start_col, start_col + cols):
            ws.cell(r, c).border = border

def add_merged_header(ws, row, start_col, col_count, title):
    ws.merge_cells(
        start_row=row,
        start_column=start_col,
        end_row=row,
        end_column=start_col + col_count - 1
    )
    cell = ws.cell(row, start_col, title)
    cell.font = header_font
    cell.alignment = center_align

# ================= CORE LOGIC =================
def create_combo_summary(df):
    companies = df["Combo"].dropna().str.split(",").explode().str.strip()
    summary = companies.value_counts().reset_index()
    summary.columns = ["Company", "Count"]

    summary.insert(0, "Sr.No.", range(1, len(summary) + 1))
    summary["Invest"] = INVESTMENT_AMOUNT
    summary["Value"]  = summary["Count"] * INVESTMENT_AMOUNT

    total_value = summary["Value"].sum()
    summary["%Alloc"] = summary["Value"] / total_value

    total_row = pd.DataFrame({
        "Sr.No.": [""],
        "Company": ["TOTAL_INVESTMENT"],
        "Count": [""],
        "Invest": [""],
        "Value": [total_value],
        "%Alloc": [""]
    })

    return pd.concat([summary, total_row], ignore_index=True)

def get_metrics_table(metrics_df, sheet):
    cols = [
        "No_of_Gains", "No_of_Loss",
        "Total_Gains_Amt", "Total_Loss_Amt",
        "Net_Gain_Loss_Amt", "%Returns"
    ]
    row = metrics_df[metrics_df["Sheet_Name"] == sheet]
    if row.empty:
        return pd.DataFrame(columns=["Metric", "Value"])

    out = row[cols].T.reset_index()
    out.columns = ["Metric", "Value"]
    return out

def get_vertical_nifty_table(nifty_df):
    if nifty_df.empty:
        return pd.DataFrame(columns=["Metric", "Value"])

    first_row = nifty_df.iloc[0]
    out = first_row.reset_index()
    out.columns = ["Metric", "Value"]
    return out

# ================= LOAD FILES =================
combo_xls  = pd.ExcelFile(input_file_1)
metrics_df = pd.read_excel(input_file_2)
nifty_xls  = pd.ExcelFile(nifty_file)

nifty_map = {normalize(s): s for s in nifty_xls.sheet_names}

# ================= WRITE OUTPUT =================
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:

    for sheet in combo_xls.sheet_names:
        df = pd.read_excel(combo_xls, sheet)
        if "Combo" not in df.columns:
            continue

        combo_table   = create_combo_summary(df)
        metrics_table = get_metrics_table(metrics_df, sheet)

        norm = normalize(sheet)
        if norm in nifty_map:
            nifty_df = pd.read_excel(nifty_xls, nifty_map[norm])
            nifty_table = get_vertical_nifty_table(nifty_df)
        else:
            nifty_table = pd.DataFrame(columns=["Metric", "Value"])

        # ========== LAYOUT ==========
        col_combo   = 1
        col_metrics = col_combo + len(combo_table.columns) + 2
        col_nifty   = col_metrics

        row_combo   = 3
        row_metrics = 3
        row_nifty   = row_metrics + len(metrics_table) + 4

        # ========== WRITE TABLES ==========
        combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
        metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
        nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)

        ws = writer.book[sheet]

        # ========== HEADERS ==========
        add_merged_header(ws, 1, col_combo, len(combo_table.columns), sheet)
        add_merged_header(ws, 1, col_metrics, len(metrics_table.columns), "METRICS")
        add_merged_header(ws, row_nifty-1, col_nifty, len(nifty_table.columns), "NIFTY_SUMMARY")

        # ========== BORDERS ==========
        apply_borders(ws, row_combo, col_combo, len(combo_table)+1, len(combo_table.columns))
        apply_borders(ws, row_metrics, col_metrics, len(metrics_table)+1, len(metrics_table.columns))
        apply_borders(ws, row_nifty, col_nifty, len(nifty_table)+1, len(nifty_table.columns))

        # ========== TOTAL HIGHLIGHT ==========
        for r in range(row_combo+1, row_combo + len(combo_table)+1):
            if ws.cell(r, col_combo+1).value == "TOTAL_INVESTMENT":
                for c in range(col_combo, col_combo + len(combo_table.columns)):
                    ws.cell(r, c).fill = total_fill
                    ws.cell(r, c).font = bold_font

print(" Excel created successfully.")

Enter FIRST input Excel file (multi-sheet combos): C:\swarupa\performance_gain_loss\Top_10_Combos\Combo_Top_10_Gain_Loss.xlsx
Enter SECOND input Excel file (METRICS): C:\Users\Swarupa\Downloads\Top_10_summary_202week.xlsx
Enter NIFTY Excel file (multi-sheet): C:\swarupa\performance_gain_loss\Top_10_Combos\NIFTY_performace_multiple_sheets_202.xlsx
Enter output Excel file path: C:\swarupa\performance_gain_loss\Top_10_Combos\METRIC_PERFORMACE_202_WEEKS.xlsx


C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarnin

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarnin

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarnin

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarnin

...

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarnin

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarnin

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarnin

C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  combo_table.to_excel(writer, sheet, index=False, startrow=row_combo-1, startcol=col_combo-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:126: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  metrics_table.to_excel(writer, sheet, index=False, startrow=row_metrics-1, startcol=col_metrics-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:127: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  nifty_table.to_excel(writer, sheet, index=False, startrow=row_nifty-1, startcol=col_nifty-1)
C:\Users\Swarupa\AppData\Local\Temp\2\ipykernel_13368\1045982550.py:125: FutureWarnin

 Excel created successfully.


In [ ]:
#Equity_Curve_dates

import pandas as pd
from dateutil.relativedelta import relativedelta

# ================= USER INPUT =================
input_file  = r"C:\swarupa\Trading Stratergies\50 comp data (input file).xlsx"
output_file = r"C:\swarupa\performance_gain_loss\Equity Curve\Weekly_Rolling_Dates.xlsx"

# ================= LOAD EXCEL =================
xls = pd.ExcelFile(input_file)

week_ranges = []

# ================= PROCESS ALL SHEETS =================
for sheet_name in xls.sheet_names:
    df = pd.read_excel(input_file, sheet_name=sheet_name)

    if 'DATE' not in df.columns:
        continue

    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.sort_values('DATE').reset_index(drop=True)

    # ---- Skip first 3 months ----
    first_date = df['DATE'].iloc[0]
    skip_date = first_date + relativedelta(months=3)
    df = df[df['DATE'] >= skip_date]

    if df.empty:
        continue

    start_date = df['DATE'].iloc[0]
    end_date   = df['DATE'].iloc[-1]

    current_start = start_date

    while current_start <= end_date:
        current_end = min(current_start + pd.Timedelta(days=6), end_date)
        week_ranges.append((current_start.date(), current_end.date()))
        current_start += pd.Timedelta(days=7)

# ================= PREPARE DATE ROW =================
date_row = []
for start, end in week_ranges:
    date_row.extend([f"Start_{start}", f"End_{end}"])

date_df = pd.DataFrame([date_row])

# ================= WRITE TO EXCEL =================
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    date_df.to_excel(
        writer,
        sheet_name="Weekly_Data",
        index=False,
        header=False,
        startrow=1   
    )

    workbook  = writer.book
    worksheet = writer.sheets["Weekly_Data"]

    # ---- Merge WK headers  ----
    col = 0
    for i in range(len(week_ranges)):
        worksheet.merge_range(0, col, 0, col + 1, f"WK_{i+1}")
        col += 2

    # Optional formatting
    worksheet.set_row(0, None, workbook.add_format({"align": "center", "bold": True}))
    worksheet.set_row(1, None, workbook.add_format({"align": "center"}))
    worksheet.set_column(0, col, 18)

print(" Excel created correctly with WK headers and date values.")


In [ ]:
#Equity_Curve_Combo_with_Dates

import pandas as pd
from openpyxl import load_workbook

# ================= USER INPUT =================
file_1 = input("Enter first Excel file path (multiple sheets): ").strip()
file_2 = input("Enter second Excel file path (single-sheet template): ").strip()
output_file = input("Enter output Excel file path: ").strip()

# ================= READ ALL COMBOS FROM FILE 1 =================
all_sheets = pd.read_excel(file_1, sheet_name=None)

rows = []

for sheet_name, df in all_sheets.items():
    if 'Combo' not in df.columns:
        continue

    combos = df['Combo'].dropna().tolist()

    for i, combo in enumerate(combos):
        rows.append([
            sheet_name if i == 0 else '',
            combo
        ])

# ================= LOAD SECOND FILE =================
wb = load_workbook(file_2)
ws = wb.active   

# ================= INSERT TWO COLUMNS BEFORE WK_1 =================
ws.insert_cols(1, 2)

# ================= ADD HEADERS =================
ws['A1'] = 'Sheet_Name'
ws['B1'] = 'Combo'

# ================= WRITE DATA =================
start_row = 3  

for i, row in enumerate(rows):
    ws.cell(row=start_row + i, column=1).value = row[0] 
    ws.cell(row=start_row + i, column=2).value = row[1]  

# ================= SAVE OUTPUT =================
wb.save(output_file)

print(" output created ")


In [ ]:
#final_quantity

import pandas as pd
import re

# ================= USER INPUT =================
combo_file  = r"C:\swarupa\performance_gain_loss\Equity Curve\final_quantity.xlsx"
price_file  = r"C:\swarupa\Trading Stratergies\50 comp data (input file).xlsx"
output_file = r"C:\swarupa\performance_gain_loss\Equity Curve\final_quantity_output.xlsx"

INVEST_PER_COMPANY = 20000

# ================= LOAD FILES =================
combo_df = pd.read_excel(combo_file)
price_df = pd.read_excel(price_file)

# --- Prepare price data ---
price_df['DATE'] = pd.to_datetime(price_df['DATE'])
price_df.set_index('DATE', inplace=True)

# ================= FIND START DATE COLUMNS =================
start_cols = [
    col for col in combo_df.columns
    if col.startswith("Start_")
]

# ================= QUANTITY CALCULATION =================
for col in start_cols:

    # Extract date safely from column name
    match = re.search(r'Start_(\d{4}-\d{2}-\d{2})', col)
    if not match:
        continue

    start_date = pd.to_datetime(match.group(1))

    new_values = []

    for _, row in combo_df.iterrows():

        companies = [c.strip() for c in row['Combo'].split(',')]
        quantities = []

        if start_date in price_df.index:
            for company in companies:
                if company in price_df.columns:
                    price = price_df.at[start_date, company]
                    if pd.notna(price) and price > 0:
                        qty = round(INVEST_PER_COMPANY / price, 2)
                        quantities.append(str(qty))
                    else:
                        quantities.append("")
                else:
                    quantities.append("")
        else:
            quantities = [""] * len(companies)

        # Put all 4 quantities in ONE cell 
        new_values.append(",".join(quantities))

    # Write ONLY to Start-date column
    combo_df[col] = new_values

# ================= SAVE OUTPUT =================
combo_df.to_excel(output_file, index=False)

print(" Quantities calculated successfully")
print(" Start columns updated")
print(" End columns unchanged")


In [ ]:
#Equity_Curve_Excel

import pandas as pd
import numpy as np
import re

# ================= FILES =================
IINPUT_FILE  = r"C:\swarupa\performance_gain_loss\Equity Curve\final_quantity_output.xlsx"
PRICE_FILE  = r"C:\swarupa\Trading Stratergies\50 comp data (input file).xlsx"
OUTPUT_FILE = r"C:\swarupa\performance_gain_loss\Equity Curve\equity_curve_excel.xlsx"


START_PER_SUBGROUP = 80000         
SUBGROUPS_PER_WEEK = 10
START_TOTAL = START_PER_SUBGROUP * SUBGROUPS_PER_WEEK  
MAX_ROWS = 6

# ================= LOAD DATA =================
raw_df = pd.read_excel(INPUT_FILE)
price_df = pd.read_excel(PRICE_FILE)

price_df["DATE"] = pd.to_datetime(price_df["DATE"])
price_df = price_df.sort_values("DATE").set_index("DATE")

ALL_COLS = raw_df.columns.tolist() + ["Buffer"]

# ================= PRICE LOOKUP =================
def get_price(company, date):
    if company not in price_df.columns:
        return np.nan
    idx = price_df.index.searchsorted(date, side="right") - 1
    return price_df.iloc[idx][company] if idx >= 0 else np.nan

# ================= DETECT START COLUMNS =================
start_cols = sorted(
    [c for c in raw_df.columns if c.startswith("Start_")],
    key=lambda x: pd.to_datetime(x.split("_")[1])
)

BASE_START = pd.to_datetime(start_cols[0].split("_")[1])
TOTAL_WEEKS = len(start_cols)

# ================= PRECOMPUTE WEEK DATA  =================

week_data = {}

for w in range(1, TOTAL_WEEKS + 1):

    wk_start = BASE_START + pd.Timedelta(days=7 * (w - 1))
    wk_end   = wk_start + pd.Timedelta(days=6)
    sc = f"Start_{wk_start.date()}"

    # ONLY 10 combos of THIS WEEK
    week_df = raw_df[raw_df["Sheet_Name"] == f"Common_{w}_45Days"]

    qty_rows = []
    end_val = 0.0

    for _, r in week_df.iterrows():

        cell = r.get(sc, np.nan)
        if pd.isna(cell):
            continue

        companies = [c.strip() for c in r["Combo"].split(",")]
        qtys = [float(x) for x in str(cell).split(",") if x.strip()]

        qty_rows.append((companies, qtys))

        for c, q in zip(companies, qtys):
            p = get_price(c, wk_end)
            if not np.isnan(p):
                end_val += q * p

    week_data[w] = {
        "qty": qty_rows,           
        "end": round(end_val, 2)
    }

# ================= BUILD OUTPUT =================
output_rows = []

for w in range(1, TOTAL_WEEKS + 1):

    wkN_start = BASE_START + pd.Timedelta(days=7 * (w - 1))
    wkN_end   = wkN_start + pd.Timedelta(days=6)

    sc = f"Start_{wkN_start.date()}"
    ec = f"End_{wkN_end.date()}"

    min_w = max(1, w - MAX_ROWS + 1)

    for idx, src_w in enumerate(range(w, min_w - 1, -1)):

        row = {c: np.nan for c in ALL_COLS}
        row["Sheet_Name"] = f"WK_{w}"
        row["Combo"] = "TOTAL"

        # ----- START VALUE -----
        if idx == 0:
            start_val = START_TOTAL
        else:
            start_val = week_data[src_w]["end"]

        row[sc] = start_val

        # ----- END VALUE -----
        # quantities from src_w, prices from wkN_end
        end_val = 0.0
        for companies, qtys in week_data[src_w]["qty"]:
            for c, q in zip(companies, qtys):
                p = get_price(c, wkN_end)
                if not np.isnan(p):
                    end_val += q * p

        end_val = round(end_val, 2)
        row[ec] = end_val

        # ----- BUFFER -----
        row["Buffer"] = round(end_val - start_val, 2)

        output_rows.append(row)

# ================= SAVE =================
final_df = pd.DataFrame(output_rows, columns=ALL_COLS)
final_df.to_excel(OUTPUT_FILE, index=False)

print(" FINAL STRICT-WEEK OUTPUT GENERATED SUCCESSFULLY")


In [2]:
#Equity_Curve_Plotting

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# ================= USER INPUT =================
INPUT_FILE = r"C:\swarupa\performance_gain_loss\Equity Curve\equity_curve_excel.xlsx"
OUTPUT_PDF = r"C:\swarupa\performance_gain_loss\Equity Curve\Equity_Curve_6Week_Rolling_WITH_BUFFER_LABELS.pdf"
WINDOW = 6
BUFFER_COL = "Buffer"   

# ================= LOAD DATA =================
df = pd.read_excel(INPUT_FILE)

# ================= DETECT START / END PAIRS =================
cols = list(df.columns)
wk_pairs = []
i = 0
while i < len(cols) - 1:
    if str(cols[i]).startswith("Start_") and str(cols[i + 1]).startswith("End_"):
        wk_pairs.append((cols[i], cols[i + 1]))
        i += 2
    else:
        i += 1

# ================= BUILD WK BLOCKS & BUFFERS =================
wk_blocks = []
wk_buffers = []

for start_col, end_col in wk_pairs:
    block = df[[start_col, end_col]].dropna(how="all").reset_index(drop=True)
    wk_blocks.append(block)

    if BUFFER_COL in df.columns:
        buf_series = df.loc[
            df[start_col].notna() | df[end_col].notna(),
            BUFFER_COL
        ].dropna()
        wk_buffers.append(buf_series.iloc[0] if not buf_series.empty else None)
    else:
        wk_buffers.append(None)

# ================= PLOTTING =================
with PdfPages(OUTPUT_PDF) as pdf:

    for win_start in range(len(wk_blocks) - WINDOW + 1):

        blocks = wk_blocks[win_start: win_start + WINDOW]
        buffers = wk_buffers[win_start: win_start + WINDOW]

        x_labels = [wk_pairs[win_start + i][1] for i in range(WINDOW)]
        x_dates = [pd.to_datetime(c.replace("End_", "")) for c in x_labels]

        fig, (ax_top, ax_bot) = plt.subplots(
            2, 1, figsize=(13, 9),
            gridspec_kw={"height_ratios": [3, 1]},
            sharex=True
        )

        # ================= TOP: EQUITY CURVES =================
        for line in range(WINDOW):

            y_vals = []
            x_vals = []

            # ---- START ----
            if line < len(blocks[line]):
                y_vals.append(blocks[line].iloc[0, 0])
                x_vals.append(x_dates[line])
            else:
                continue

            # ---- END  ----
            for wk in range(line + 1, WINDOW):
                if len(blocks[wk]) == 0:
                    continue
                row_idx = min(line, len(blocks[wk]) - 1)
                y_vals.append(blocks[wk].iloc[row_idx, 1])
                x_vals.append(x_dates[wk])

            ax_top.plot(
                x_vals,
                y_vals,
                marker="o",
                linewidth=2,
                label=f"Line-{line+1} (Start WK-{win_start+line+1})"
            )

        ax_top.set_title(
            f"6-Week Rolling Equity Curve (WK {win_start+1} → WK {win_start+WINDOW})"
        )
        ax_top.set_ylabel("Portfolio Value")
        ax_top.grid(True)
        ax_top.legend(loc="upper left", fontsize=9)

        # ================= BOTTOM: BUFFER =================
        buf_x = []
        buf_y = []

        for i, val in enumerate(buffers):
            if val is not None:
                buf_x.append(x_dates[i])
                buf_y.append(val)

        ax_bot.plot(
            buf_x,
            buf_y,
            marker="o",
            linewidth=2,
            color="black",
            label="Buffer"
        )
        ax_bot.axhline(
            0,
            linestyle="--",
            linewidth=1,
            label="Zero Line"
        )

        ax_bot.set_ylabel("Buffer")
        ax_bot.set_xlabel("Week (Excel End Date Headers)")
        ax_bot.grid(True)
        ax_bot.legend(loc="upper left", fontsize=9)

        ax_bot.set_xticks(x_dates)
        ax_bot.set_xticklabels(x_labels, rotation=45)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(" Equity curves + Buffer generated successfully")
print(f" Output file: {OUTPUT_PDF}")


 Equity curves + Buffer generated successfully
 Output file: C:\swarupa\performance_gain_loss\Equity Curve\Equity_Curve_6Week_Rolling_WITH_BUFFER_LABELS.pdf
